# Start

In [1]:
hk_tickers = ["0001.HK", "0002.HK", "0003.HK", "0005.HK", "0006.HK", "0011.HK", '0012.HK', '0016.HK', '0027.HK', '0066.HK', '0101.HK', '0175.HK', '0241.HK', '0267.HK', '0285.HK', '0288.HK', '0291.HK', '0316.HK', 
                  "0322.HK", '0386.HK', '0388.HK', '0669.HK', '0688.HK', '0700.HK', '0762.HK', '0823.HK', '0836.HK', '0857.HK', '0868.HK', '0881.HK', '0883.HK', '0939.HK', '0941.HK', '0960.HK', '0968.HK', '0981.HK', 
                  "0992.HK", "1024.HK", '1038.HK', '1044.HK', '1088.HK', "1093.HK", '1099.HK', '1109.HK', '1113.HK', '1177.HK', '1209.HK', '1211.HK', '1299.HK', '1378.HK', '1398.HK', '1810.HK', '1876.HK', '1928.HK',
                  '1929.HK', '1997.HK', '2015.HK', '2020.HK', '2269.HK', '2313.HK', '2318.HK', '2319.HK', '2331.HK', '2359.HK', '2382.HK', '2388.HK', '2628.HK', '2688.HK', '2899.HK', '3690.HK', '3692.HK', '3968.HK', 
                  '3988.HK', '6618.HK', '6690.HK', '6862.HK', '9618.HK', '9633.HK', '9888.HK', '9901.HK', '9961.HK', '9988.HK', '9999.HK']
len(hk_tickers)

83

# Human review/ correction (if there is a minor mistake from LLM output)

In [1]:
import pandas as pd
import io

# Define the filename
filename = 'HK_final_comprehensive_result_3.csv'

try:
    # --- Load the CSV file ---
    # In your actual use, replace the io.StringIO part with just the filename:
    df = pd.read_csv(filename)
    # df = pd.read_csv(io.StringIO(csv_data))
    print(f"Successfully loaded '{filename}'. Original rows: {len(df)}")
    # print("Original DataFrame head:\n", df.head()) # Optional: view data

    # --- Identify and remove the duplicate based on 'code' ---
    # We keep the 'first' occurrence and remove subsequent ones
    # based on the 'domain' and 'code' columns being identical.
    # This handles the specific case of row 1 being a duplicate of row 0
    # based on these columns.
    original_rows = len(df)
    df_deduplicated = df.drop_duplicates(subset=['domain', 'code'], keep='first')
    rows_removed = original_rows - len(df_deduplicated)

    if rows_removed > 0:
        print(f"Removed {rows_removed} duplicate row(s) based on 'domain' and 'code'.")
    else:
        print("No duplicate rows found based on 'domain' and 'code'.")

    # --- Save the modified DataFrame back to the exact same file ---
    # index=False prevents pandas from writing the DataFrame index as a column
    df_deduplicated.to_csv(filename, index=False)
    print(f"Successfully saved the updated data back to '{filename}'. Final rows: {len(df_deduplicated)}")
    # print("\nFinal DataFrame head:\n", df_deduplicated.head()) # Optional: view updated data

except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded 'HK_final_comprehensive_result_3.csv'. Original rows: 16
Removed 1 duplicate row(s) based on 'domain' and 'code'.
Successfully saved the updated data back to 'HK_final_comprehensive_result_3.csv'. Final rows: 15


# Alpha factor calculation and importing relevant library for later use

In [2]:
!pip install ta

In [3]:
import pandas as pd
import numpy as np
import ta # Still used for ATR, RSI initially
from datetime import datetime, timedelta
import yfinance as yf

# Helper function to ensure required columns exist
def _check_columns(df):
    required_cols = {'Open', 'High', 'Low', 'Close', 'Volume'}
    if not required_cols.issubset(df.columns):
        missing_cols = required_cols - set(df.columns)
        raise ValueError(f"Input DataFrame missing required columns: {missing_cols}. Available columns: {list(df.columns)}")

# --------------------------------------------------
# Alpha Factor Calculation Functions (Manual Pandas Implementation)
# Using 'Close' column name
# --------------------------------------------------

# == Momentum Domain ==
def calculate_price_momentum_10d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    close_delayed = df[price_col].shift(10)
    momentum = (df[price_col] - close_delayed) / close_delayed
    return momentum.replace([np.inf, -np.inf], np.nan) * 100

def calculate_ma_crossover_10_50(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    sma10 = df[price_col].rolling(window=10, min_periods=10).mean()
    sma50 = df[price_col].rolling(window=50, min_periods=50).mean()
    return sma10 - sma50

def calculate_volume_momentum_50d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df)
    return df['Volume'] - df['Volume'].shift(50)

def calculate_roc_50d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    close_delayed = df[price_col].shift(50)
    roc = ((df[price_col] - close_delayed) / close_delayed) * 100
    return roc.replace([np.inf, -np.inf], np.nan)

# == Mean Reversion Domain ==
def calculate_mean_reversion_20d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    sma20 = df[price_col].rolling(window=20, min_periods=20).mean()
    return sma20 - df[price_col]

def calculate_moving_average_reversion(df: pd.DataFrame) -> pd.Series:
    # Identical to mean_reversion_20d
    return calculate_mean_reversion_20d(df)

def calculate_stochastic_oscillator_14d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    low_14 = df['Low'].rolling(window=14, min_periods=14).min()
    high_14 = df['High'].rolling(window=14, min_periods=14).max()
    denominator = high_14 - low_14
    stoch_k = 100 * (df[price_col] - low_14) / denominator.replace(0, np.nan)
    return stoch_k.replace([np.inf, -np.inf], np.nan)

# == Volatility Domain ==
def calculate_atr_14d(df: pd.DataFrame) -> pd.Series:
    # Keeping ta for now for ATR
    _check_columns(df);
    print("Calculating ATR using 'ta' library...") # Add trace
    try:
        atr = ta.volatility.AverageTrueRange(
            high=df['High'], low=df['Low'], close=df['Close'], window=14
        ).average_true_range()
    except Exception as e:
        print(f"Error calculating ATR with ta: {e}")
        atr = pd.Series(np.nan, index=df.index)
    return atr

def calculate_daily_high_low_range(df: pd.DataFrame) -> pd.Series:
    _check_columns(df)
    return df['High'] - df['Low']

def calculate_normalized_bollinger_band_width_20d_2std(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    sma20 = df[price_col].rolling(window=20, min_periods=20).mean()
    std20 = df[price_col].rolling(window=20, min_periods=20).std(ddof=1)
    bbw_normalized = (4 * std20) / sma20
    return bbw_normalized.replace([np.inf, -np.inf], np.nan)

# == Liquidity Domain ==
def calculate_volume_roc_10d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df)
    vol_delayed = df['Volume'].shift(10)
    vroc = ((df['Volume'] / vol_delayed) - 1) * 100
    return vroc.replace([np.inf, -np.inf], np.nan)

# == Technical Domain ==
def calculate_sma_20d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    sma20 = df[price_col].rolling(window=20, min_periods=20).mean()
    return sma20

def calculate_ema_20d(df: pd.DataFrame) -> pd.Series:
    _check_columns(df); price_col = 'Close'
    ema20 = df[price_col].ewm(span=20, adjust=False, min_periods=20).mean()
    return ema20

def calculate_rsi_14d(df: pd.DataFrame) -> pd.Series:
    # Keeping ta for now for RSI
    _check_columns(df); price_col = 'Close'
    print("Calculating RSI using 'ta' library...") # Add trace
    try:
        rsi = ta.momentum.RSIIndicator(close=df[price_col], window=14).rsi()
    except Exception as e:
        print(f"Error calculating RSI with ta: {e}")
        rsi = pd.Series(np.nan, index=df.index)
    return rsi

# --------------------------------------------------
# Main Calculation Function
# --------------------------------------------------

def calculate_all_alpha_factors(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    # Momentum
    df_out['Price_Momentum_10d'] = calculate_price_momentum_10d(df_out)
    # df_out['ROC_10d'] = calculate_roc_10d(df_out) # REMOVED
    df_out['MA_Crossover_10_50'] = calculate_ma_crossover_10_50(df_out)
    df_out['Volume_Momentum_50d'] = calculate_volume_momentum_50d(df_out)
    df_out['ROC_50d'] = calculate_roc_50d(df_out)
    # Mean Reversion
    df_out['Mean_Reversion_20d'] = calculate_mean_reversion_20d(df_out)
    df_out['Moving_Average_Reversion'] = calculate_moving_average_reversion(df_out)
    df_out['Stochastic_Oscillator_14d'] = calculate_stochastic_oscillator_14d(df_out)
    # Volatility
    df_out['ATR_14d'] = calculate_atr_14d(df_out) # Still uses ta
    df_out['Daily_High_Low_Range'] = calculate_daily_high_low_range(df_out)
    df_out['Normalized_BBW_20d_2std'] = calculate_normalized_bollinger_band_width_20d_2std(df_out)
    # Liquidity
    df_out['VROC_10d'] = calculate_volume_roc_10d(df_out)
    # Adding Trading Volume as a factor is handled *after* this function call
    # by renaming the existing 'Volume' column before final selection.
    # Technical
    df_out['SMA_20d'] = calculate_sma_20d(df_out)
    df_out['EMA_20d'] = calculate_ema_20d(df_out)
    df_out['RSI_14d'] = calculate_rsi_14d(df_out) # Still uses ta
    return df_out

# --------------------------------------------------
# Example Usage
# --------------------------------------------------

if __name__ == "__main__":
    tickers = ["0001.HK", "0002.HK", "0003.HK", "0005.HK", "0006.HK", "0011.HK", '0012.HK', '0016.HK', '0027.HK', '0066.HK', '0101.HK', '0175.HK', '0241.HK', '0267.HK', '0285.HK', '0288.HK', '0291.HK', '0316.HK', 
                  "0322.HK", '0386.HK', '0388.HK', '0669.HK', '0688.HK', '0700.HK', '0762.HK', '0823.HK', '0836.HK', '0857.HK', '0868.HK', '0881.HK', '0883.HK', '0939.HK', '0941.HK', '0960.HK', '0968.HK', '0981.HK', 
                  "0992.HK", "1024.HK", '1038.HK', '1044.HK', '1088.HK', "1093.HK", '1099.HK', '1109.HK', '1113.HK', '1177.HK', '1209.HK', '1211.HK', '1299.HK', '1378.HK', '1398.HK', '1810.HK', '1876.HK', '1928.HK',
                  '1929.HK', '1997.HK', '2015.HK', '2020.HK', '2269.HK', '2313.HK', '2318.HK', '2319.HK', '2331.HK', '2359.HK', '2382.HK', '2388.HK', '2628.HK', '2688.HK', '2899.HK', '3690.HK', '3692.HK', '3968.HK', 
                  '3988.HK', '6618.HK', '6690.HK', '6862.HK', '9618.HK', '9633.HK', '9888.HK', '9901.HK', '9961.HK', '9988.HK', '9999.HK']
    start_date = datetime.strptime("2020-03-24",'%Y-%m-%d')
    end_date = datetime.strptime("2025-03-24",'%Y-%m-%d')

    processed_data_list = []
    alpha_factor_columns = None

    # Calculate the end_date for yfinance download (exclusive)
    yf_end_date = end_date + timedelta(days=1)

    print(f"Fetching and processing data for {tickers} from {start_date} to {end_date}...")
    for ticker in tickers:
        data = None
        try:
            data = yf.download(ticker, start=start_date, end=yf_end_date,
                               auto_adjust=True, progress=False)

            if data.empty:
                print(f"No data found for {ticker} in the specified date range.")
                continue

            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)

            print(f"Calculating alpha factors for {ticker}...")
            alpha_data = calculate_all_alpha_factors(data.copy())
            print(f"Finished calculating for {ticker}.")

            # --- Post-processing for final DataFrame format ---
            alpha_data['stock_id'] = ticker
            alpha_data.reset_index(inplace=True)

            # Handle Date column naming
            date_col_name = None
            if 'Date' in alpha_data.columns: date_col_name = 'Date'
            elif 'Datetime' in alpha_data.columns: date_col_name = 'Datetime'; alpha_data.rename(columns={'Datetime': 'Date'}, inplace=True)
            elif 'index' in alpha_data.columns: date_col_name = 'index'; alpha_data.rename(columns={'index': 'Date'}, inplace=True)
            else: print(f"Warning: Could not find Date column for {ticker}. Skipping."); continue

            # --- ADD TRADING VOLUME ---
            # Explicitly rename 'Volume' to 'Trading_Volume' so it's treated as a factor
            if 'Volume' in alpha_data.columns:
                alpha_data.rename(columns={'Volume': 'Trading_Volume'}, inplace=True)
            # --- End Add Trading Volume ---


            # Dynamically identify alpha factor columns (only once)
            if alpha_factor_columns is None:
                # Define original cols expected from yfinance *before* rename + added ones
                # 'Volume' is no longer original, it's renamed to 'Trading_Volume'
                original_cols = {'Open', 'High', 'Low', 'Close', 'Date', 'stock_id'}
                alpha_factor_columns = sorted([col for col in alpha_data.columns if col not in original_cols])
                # Ensure 'Trading_Volume' is captured if the rename happened
                if 'Trading_Volume' not in alpha_factor_columns and 'Trading_Volume' in alpha_data.columns:
                     alpha_factor_columns.append('Trading_Volume')
                     alpha_factor_columns.sort()


            # Select only the necessary columns for the final DF
            columns_to_keep_final = ['Date', 'stock_id'] + alpha_factor_columns
            alpha_data_filtered = alpha_data[[col for col in columns_to_keep_final if col in alpha_data.columns]].copy()

            processed_data_list.append(alpha_data_filtered)
            # --- End of Post-processing ---

        except ValueError as ve:
             print(f"Could not process data for {ticker}: {ve}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {ticker}: {e}")


    # --- Combine and Finalize ---
    if processed_data_list:
        print("\nCombining dataframes...")
        final_df = pd.concat(processed_data_list, ignore_index=True)

        final_df['Date'] = pd.to_datetime(final_df['Date'])
        final_df.sort_values(by=['stock_id', 'Date'], inplace=True)
        final_df.reset_index(drop=True, inplace=True)

        print("\n--- Final Combined DataFrame ---")
        print(f"Shape: {final_df.shape}")
        print("\n--- Sample (First 10 rows) ---")
        print(final_df.head(10))
        print("\n--- Sample (Last 10 rows) ---")
        print(final_df.tail(10))
        print("\n--- DataFrame Info ---")
        final_df.info()
    else:
        print("\nNo data processed, cannot create final DataFrame.")

Fetching and processing data for ['0001.HK', '0002.HK', '0003.HK', '0005.HK', '0006.HK', '0011.HK', '0012.HK', '0016.HK', '0027.HK', '0066.HK', '0101.HK', '0175.HK', '0241.HK', '0267.HK', '0285.HK', '0288.HK', '0291.HK', '0316.HK', '0322.HK', '0386.HK', '0388.HK', '0669.HK', '0688.HK', '0700.HK', '0762.HK', '0823.HK', '0836.HK', '0857.HK', '0868.HK', '0881.HK', '0883.HK', '0939.HK', '0941.HK', '0960.HK', '0968.HK', '0981.HK', '0992.HK', '1024.HK', '1038.HK', '1044.HK', '1088.HK', '1093.HK', '1099.HK', '1109.HK', '1113.HK', '1177.HK', '1209.HK', '1211.HK', '1299.HK', '1378.HK', '1398.HK', '1810.HK', '1876.HK', '1928.HK', '1929.HK', '1997.HK', '2015.HK', '2020.HK', '2269.HK', '2313.HK', '2318.HK', '2319.HK', '2331.HK', '2359.HK', '2382.HK', '2388.HK', '2628.HK', '2688.HK', '2899.HK', '3690.HK', '3692.HK', '3968.HK', '3988.HK', '6618.HK', '6690.HK', '6862.HK', '9618.HK', '9633.HK', '9888.HK', '9901.HK', '9961.HK', '9988.HK', '9999.HK'] from 2020-03-24 00:00:00 to 2025-03-24 00:00:00...
Ca

In [4]:
final_df

Price,Date,stock_id,ATR_14d,Daily_High_Low_Range,EMA_20d,MA_Crossover_10_50,Mean_Reversion_20d,Moving_Average_Reversion,Normalized_BBW_20d_2std,Price_Momentum_10d,ROC_50d,RSI_14d,SMA_20d,Stochastic_Oscillator_14d,Trading_Volume,VROC_10d,Volume_Momentum_50d
0,2020-03-24,0001.HK,0.000000,1.379356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15296458,NaN,NaN
1,2020-03-25,0001.HK,0.000000,1.034517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16640222,NaN,NaN
2,2020-03-26,0001.HK,0.000000,1.685881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12067696,NaN,NaN
3,2020-03-27,0001.HK,0.000000,1.417674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18221966,NaN,NaN
4,2020-03-30,0001.HK,0.000000,1.455989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15936885,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100020,2025-03-18,9999.HK,5.256375,2.800003,159.681671,4.432549,-0.407722,-0.407722,0.083457,2.141484,16.265853,50.669154,159.092278,61.023137,6361971,-9.442467,2009660.0
100021,2025-03-19,9999.HK,5.238063,5.000000,159.731035,4.174289,-1.210040,-1.210040,0.081710,1.392403,17.546056,51.706898,158.989957,65.157025,6040521,20.028363,-24156.0
100022,2025-03-20,9999.HK,5.178202,4.400009,159.442365,3.022003,2.232294,2.232294,0.082528,-4.625688,14.308009,46.445822,158.932291,44.487496,5602934,-26.169430,3315930.0
100023,2025-03-21,9999.HK,5.201187,4.900009,158.724044,1.759711,6.639780,6.639780,0.091520,-6.002483,10.645385,40.378045,158.539774,16.140695,7816314,-34.458375,1515005.0


In [5]:
final_df.to_csv("alpha_data_values_1.csv")

In [6]:
!pip install pandas numpy scipy matplotlib statsmodels yfinance tqdm

In [7]:
!pip install pandas_market_calendars

In [8]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install openpyxl

# REAL

In [10]:
alpha_factors_df = pd.read_csv('alpha_data_values_1.csv')
alpha_factor_names = alpha_factors_df.drop(columns=['Date', 'stock_id'], errors='ignore').columns[1:].to_list()
alpha_factor_names

['ATR_14d',
 'Daily_High_Low_Range',
 'EMA_20d',
 'MA_Crossover_10_50',
 'Mean_Reversion_20d',
 'Moving_Average_Reversion',
 'Normalized_BBW_20d_2std',
 'Price_Momentum_10d',
 'ROC_50d',
 'RSI_14d',
 'SMA_20d',
 'Stochastic_Oscillator_14d',
 'Trading_Volume',
 'VROC_10d',
 'Volume_Momentum_50d']

In [11]:
# Change it to MultiIndex
alpha_factors_df = pd.read_csv('alpha_data_values_1.csv')
alpha_factor_names = alpha_factors_df.drop(columns=['Date', 'stock_id'], errors='ignore').columns[1:].to_list()
alpha_factors_df = alpha_factors_df.rename(columns={'Date': 'date', 'stock_id': 'asset'})
alpha_factors = pd.DataFrame()

alpha_data = {}
for factor_name in alpha_factor_names:
    """
    pivot_df = alpha_factors_df.pivot_table(index='date', columns='asset', values=factor_name)
    pivot_df = pivot_df.reindex(dates)  # Reindex to ensure all dates are present
    
    # Create the MultiIndex and stack the pivoted DataFrame
    multi_index = pd.MultiIndex.from_product([pivot_df.index, pivot_df.columns], names=['date', 'asset'])
    stacked_series = pivot_df.stack().reindex(multi_index)
    
    alpha_factors[factor_name] = stacked_series"
    """
    pivot_df = alpha_factors_df.pivot_table(index='date', columns='asset', values=factor_name)

    #pivot_df = pivot_df.reindex(dates)
    #print(pivot_df)
    #break

    multi_index = pd.MultiIndex.from_product([pivot_df.index, pivot_df.columns], names=['date', 'asset'])

    stacked_series = pivot_df.stack().reindex(multi_index)

    alpha_factors[factor_name] = stacked_series

alpha_factors # Print the first few rows to verify the result.

alpha_factors.to_csv("processed_alpha_data_values_1.csv")

In [12]:
alpha_factors

ATR_14d  Daily_High_Low_Range     EMA_20d  \
date       asset                                                  
2020-03-24 0001.HK   0.000000              1.379356         NaN   
           0002.HK   0.000000              2.383914         NaN   
           0003.HK   0.000000              0.339107         NaN   
           0005.HK   0.000000              0.587237         NaN   
           0006.HK   0.000000              1.422925         NaN   
...                       ...                   ...         ...   
2025-03-24 9888.HK   4.110813              2.800003   91.777740   
           9901.HK   1.812617              3.900002   39.011644   
           9961.HK  20.539906             18.799988  497.756730   
           9988.HK   6.116542              3.700012  131.956694   
           9999.HK   5.243960              5.699997  158.521755   

                    MA_Crossover_10_50  Mean_Reversion_20d  \
date       asset                                             
2020-03-24 0001.HK                 NaN                 NaN   
           0002.HK                 NaN                 NaN   
           0003.HK                 NaN                 NaN   
           0005.HK                 NaN                 NaN   
           0006.HK                 NaN                 NaN   
...                                ...                 ...   
2025-03-24 9888.HK            6.512000           -2.932502   
           9901.HK           -1.117000            0.920000   
           9961.HK          -14.091614          -27.012465   
           9988.HK           22.201001            1.639997   
           9999.HK            0.935292            1.912050   

                    Moving_Average_Reversion  Normalized_BBW_20d_2std  \
date       asset                                                        
2020-03-24 0001.HK                       NaN                      NaN   
           0002.HK                       NaN                      NaN   
           0003.HK                       NaN                      NaN   
           0005.HK                       NaN                      NaN   
           0006.HK                       NaN                      NaN   
...                                      ...                      ...   
2025-03-24 9888.HK                 -2.932502                 0.213538   
           9901.HK                  0.920000                 0.173413   
           9961.HK                -27.012465                 0.199746   
           9988.HK                  1.639997                 0.132679   
           9999.HK                  1.912050                 0.091870   

                    Price_Momentum_10d    ROC_50d    RSI_14d     SMA_20d  \
date       asset                                                           
2020-03-24 0001.HK                 NaN        NaN        NaN         NaN   
           0002.HK                 NaN        NaN        NaN         NaN   
           0003.HK                 NaN        NaN        NaN         NaN   
           0005.HK                 NaN        NaN        NaN         NaN   
           0006.HK                 NaN        NaN        NaN         NaN   
...                                ...        ...        ...         ...   
2025-03-24 9888.HK            1.684786  17.157176  53.872340   90.617501   
           9901.HK           -3.125002 -23.770489  42.638680   38.120000   
           9961.HK            3.574239   0.064281  53.140591  483.487535   
           9988.HK           -1.263938  64.153276  54.166481  134.440000   
           9999.HK           -3.213842  10.766067  47.597101  158.512056   

                    Stochastic_Oscillator_14d  Trading_Volume    VROC_10d  \
date       asset                                                            
2020-03-24 0001.HK                        NaN      15296458.0         NaN   
           0002.HK                        NaN       9264804.0         NaN   
           0003.HK                        NaN      48176115.0         NaN   
           0005.HK 

In [13]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [14]:
# --- Imports ---
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
# import matplotlib.pyplot as plt # Keep commented unless plotting is explicitly re-enabled
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS # Correct import for RollingOLS
from statsmodels.tools.sm_exceptions import MissingDataError # Import specific error
from numpy.linalg import LinAlgError # Import specific error
import yfinance as yf
from datetime import timedelta
import traceback # For detailed error reporting
from tqdm import tqdm # Use standard tqdm
import warnings # To suppress specific warnings if needed
import os # For path handling
import math # For sqrt
import openpyxl # Explicitly import for ExcelWriter engine check
import time # Can be useful for adding delays

# --- Suppress Warnings ---
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning) # Suppress PerformanceWarning if desired
pd.options.mode.chained_assignment = None # Suppress SettingWithCopyWarning ('warn' or None)

# --- Configuration ---
_start_date_str = '2020-03-24'
_end_date_str = '2025-03-24'
target_timezone = 'UTC'

# Analysis Parameters
benchmark_ticker = "^HSI"
analysis_periods_str = ['1D_fwd_ret', '3D_fwd_ret', '5D_fwd_ret']
fwd_ret_periods_int = tuple(int(p.split('D')[0]) for p in analysis_periods_str)
num_quantiles = 5
ic_method = 'spearman' # 'spearman' or 'pearson'
neutralization_lookback = 60 # For style factors & neutralization lookback
MAX_DECAY_LAG = 20
# Lookbacks for style factors (keep generic longest lookback calculation)
longest_lookback_generic = neutralization_lookback # Adjust if other lookbacks needed for style/data fetch

# Output Configuration
output_dir = "factor_analysis_output_combined"
os.makedirs(output_dir, exist_ok=True)
# Define the SINGLE output Excel file name
combined_output_filename = os.path.join(output_dir, "combined_factor_analysis_results_1.xlsx")

# --- Date Handling ---
try:
    start_date_naive = pd.to_datetime(_start_date_str)
    end_date_naive = pd.to_datetime(_end_date_str)
    start_date = start_date_naive.tz_localize(target_timezone)
    end_date = end_date_naive.tz_localize(target_timezone)
    print(f"INFO: Start date localized to {target_timezone}: {start_date}")
    print(f"INFO: End date localized to {target_timezone}: {end_date}")
except Exception as e:
    print(f"ERROR: Could not localize start/end dates to timezone '{target_timezone}'. Error: {e}")
    raise ValueError("Failed to set timezone for start/end dates") from e

# --- Placeholder for Dynamic Universe ---
def get_index_constituents_historical(index_ticker, date_str):
    print(f"Attempting to get constituents for {index_ticker} on {date_str}.")
    #print("         This is a placeholder - yfinance lacks this feature.")
    #print("         Returning a default hardcoded list for now.")
    default_assets = ["0001.HK", "0002.HK", "0003.HK", "0005.HK", "0006.HK", "0011.HK", '0012.HK', '0016.HK', '0027.HK', '0066.HK', '0101.HK', '0175.HK', '0241.HK', '0267.HK', '0285.HK', '0288.HK', '0291.HK', '0316.HK', 
                  "0322.HK", '0386.HK', '0388.HK', '0669.HK', '0688.HK', '0700.HK', '0762.HK', '0823.HK', '0836.HK', '0857.HK', '0868.HK', '0881.HK', '0883.HK', '0939.HK', '0941.HK', '0960.HK', '0968.HK', '0981.HK', 
                  "0992.HK", "1024.HK", '1038.HK', '1044.HK', '1088.HK', "1093.HK", '1099.HK', '1109.HK', '1113.HK', '1177.HK', '1209.HK', '1211.HK', '1299.HK', '1378.HK', '1398.HK', '1810.HK', '1876.HK', '1928.HK',
                  '1929.HK', '1997.HK', '2015.HK', '2020.HK', '2269.HK', '2313.HK', '2318.HK', '2319.HK', '2331.HK', '2359.HK', '2382.HK', '2388.HK', '2628.HK', '2688.HK', '2899.HK', '3690.HK', '3692.HK', '3968.HK', 
                  '3988.HK', '6618.HK', '6690.HK', '6862.HK', '9618.HK', '9633.HK', '9888.HK', '9901.HK', '9961.HK', '9988.HK', '9999.HK']
    # Add more realistic tickers if possible for better testing
    # default_assets.extend(["1299.HK", "2318.HK", "0011.HK", "0003.HK", "0016.HK", "0012.HK"])
    if index_ticker == "^HSI": return default_assets
    else: return []

target_index = "^HSI"
print(f"\n--- Attempting to define universe based on index: {target_index} ---")
assets = get_index_constituents_historical(target_index, _start_date_str)
if not assets:
    print(f"ERROR: Could not determine assets for index {target_index}. Falling back to hardcoded list.")
    assets =  ["0001.HK", "0002.HK", "0003.HK", "0005.HK", "0006.HK", "0011.HK", '0012.HK', '0016.HK', '0027.HK', '0066.HK', '0101.HK', '0175.HK', '0241.HK', '0267.HK', '0285.HK', '0288.HK', '0291.HK', '0316.HK', 
                "0322.HK", '0386.HK', '0388.HK', '0669.HK', '0688.HK', '0700.HK', '0762.HK', '0823.HK', '0836.HK', '0857.HK', '0868.HK', '0881.HK', '0883.HK', '0939.HK', '0941.HK', '0960.HK', '0968.HK', '0981.HK', 
                "0992.HK", "1024.HK", '1038.HK', '1044.HK', '1088.HK', "1093.HK", '1099.HK', '1109.HK', '1113.HK', '1177.HK', '1209.HK', '1211.HK', '1299.HK', '1378.HK', '1398.HK', '1810.HK', '1876.HK', '1928.HK',
                '1929.HK', '1997.HK', '2015.HK', '2020.HK', '2269.HK', '2313.HK', '2318.HK', '2319.HK', '2331.HK', '2359.HK', '2382.HK', '2388.HK', '2628.HK', '2688.HK', '2899.HK', '3690.HK', '3692.HK', '3968.HK', 
                '3988.HK', '6618.HK', '6690.HK', '6862.HK', '9618.HK', '9633.HK', '9888.HK', '9901.HK', '9961.HK', '9988.HK', '9999.HK']

assets = sorted(list(set(assets))) # Ensure unique and sorted
print(f"INFO: Using asset universe (Count: {len(assets)}): {assets[:10]}...")

# --- Create Target Business Day Index ---
try:
    import pandas_market_calendars as mcal
    hk_calendar = mcal.get_calendar('XHKG')
    max_fwd_buffer_days = max(fwd_ret_periods_int) + 10 if fwd_ret_periods_int else 10
    # Extend fetch range slightly more for lookbacks and forward returns
    calendar_start_naive = start_date_naive - timedelta(days=longest_lookback_generic + 40) # Slightly longer buffer
    calendar_end_naive = end_date_naive + timedelta(days=max(max_fwd_buffer_days, MAX_DECAY_LAG + 15)) # Slightly longer buffer
    schedule = hk_calendar.schedule(start_date=calendar_start_naive, end_date=calendar_end_naive)
    fetch_dates_index_raw = pd.to_datetime(schedule.index).tz_localize(schedule.index.tz)
    if fetch_dates_index_raw.tz is None: fetch_dates_index_raw = pd.to_datetime(schedule.index).tz_localize('UTC', ambiguous='infer', nonexistent='shift_forward')
    fetch_dates_index = fetch_dates_index_raw.tz_convert(target_timezone).drop_duplicates().sort_values() # Ensure unique & sorted early
    dates_index = fetch_dates_index[(fetch_dates_index >= start_date) & (fetch_dates_index <= end_date)].drop_duplicates().sort_values()
    print(f"Using pandas_market_calendars for HK business days. Full fetch range index length: {len(fetch_dates_index)}")
except ImportError:
    print("WARNING: pandas_market_calendars not found. Using pd.date_range(freq='B'). This might include holidays.")
    max_fwd_period = max(fwd_ret_periods_int) if fwd_ret_periods_int else 0
    fetch_start_dt_b = start_date - pd.Timedelta(days=longest_lookback_generic + 40) # Adjusted buffer
    fetch_end_dt_b = end_date + pd.Timedelta(days=max(max_fwd_period, MAX_DECAY_LAG) + 15) # Adjusted buffer
    fetch_dates_index = pd.date_range(start=fetch_start_dt_b, end=fetch_end_dt_b, freq='B', tz=target_timezone).drop_duplicates().sort_values()
    dates_index = fetch_dates_index[(fetch_dates_index >= start_date) & (fetch_dates_index <= end_date)].drop_duplicates().sort_values()

if dates_index.empty:
    raise ValueError(f"ERROR: Target dates_index is empty after filtering between {start_date} and {end_date}.")
print(f"Target Analysis Date Range: {dates_index.min()} to {dates_index.max()} ({len(dates_index)} analysis days)")

# --- Data Fetching (Prices and Total Volume) ---
print("\n--- Downloading Price and Total Volume Data ---")
prices_lookback = pd.DataFrame()
volumes_lookback = pd.DataFrame()
benchmark_prices_lookback = pd.Series(dtype=float)
try:
    fetch_start_str = fetch_dates_index.min().strftime('%Y-%m-%d')
    fetch_end_str = (fetch_dates_index.max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d') # Add 1 day for yf end date convention
    print(f"Fetching data from {fetch_start_str} to {fetch_end_str} for {len(assets)} assets + benchmark {benchmark_ticker}...")

    # Fetch asset data
    data_assets = yf.download(assets, start=fetch_start_str, end=fetch_end_str, progress=True, timeout=180, group_by='ticker')

    # Fetch benchmark data
    data_benchmark = yf.download(benchmark_ticker, start=fetch_start_str, end=fetch_end_str, progress=False)

    # Process Asset Data
    prices_list = []
    volumes_list = []
    valid_assets = [] # Keep track of assets with successfully downloaded data
    if not data_assets.empty:
        # Check if data_assets index needs converting (can happen with yfinance sometimes)
        if not isinstance(data_assets.index, pd.DatetimeIndex):
             try: data_assets.index = pd.to_datetime(data_assets.index)
             except: print("WARN: Could not convert downloaded asset data index to DatetimeIndex.")

        for asset in assets:
            try:
                # Access asset data robustly
                if isinstance(data_assets.columns, pd.MultiIndex):
                    if asset in data_assets.columns.get_level_values(0):
                       asset_data = data_assets[asset]
                    else:
                       print(f"WARN: No data returned for {asset} in multi-index result.")
                       continue
                elif len(assets) == 1 and asset == assets[0]: # Handle case where only one asset was requested (no multi-index)
                     asset_data = data_assets
                else: # Should not happen if group_by='ticker' worked for multiple assets
                     print(f"WARN: Unexpected data structure for {asset}. Skipping.")
                     continue

                # Select price and volume
                adj_close_key = 'Adj Close' if 'Adj Close' in asset_data.columns else 'Close'
                if adj_close_key not in asset_data.columns or 'Volume' not in asset_data.columns:
                     print(f"WARN: Missing '{adj_close_key}' or 'Volume' for {asset}. Skipping.")
                     continue
                price_col = asset_data[adj_close_key]
                volume_col = asset_data['Volume']

                # Check for sufficient non-NaN data
                if not price_col.dropna().empty: # Check if not ALL NaN
                    prices_list.append(price_col.rename(asset))
                    volumes_list.append(volume_col.rename(asset))
                    valid_assets.append(asset)
                else:
                    print(f"WARN: Price data for {asset} is all NaN.")

            except KeyError:
                print(f"WARN: KeyError accessing data for {asset}. Ticker might be invalid or delisted for the period.")
            except Exception as e_asset:
                print(f"WARN: Could not process data for {asset}. Error: {e_asset}")

    if prices_list:
        prices_raw = pd.concat(prices_list, axis=1)
        volumes_raw = pd.concat(volumes_list, axis=1)

        # Convert index to datetime and localize if needed
        if not isinstance(prices_raw.index, pd.DatetimeIndex): prices_raw.index = pd.to_datetime(prices_raw.index)
        if prices_raw.index.tz is None: prices_raw.index = prices_raw.index.tz_localize('UTC', ambiguous='infer', nonexistent='shift_forward')
        prices_raw = prices_raw.tz_convert(target_timezone)

        if not isinstance(volumes_raw.index, pd.DatetimeIndex): volumes_raw.index = pd.to_datetime(volumes_raw.index)
        if volumes_raw.index.tz is None: volumes_raw.index = volumes_raw.index.tz_localize('UTC', ambiguous='infer', nonexistent='shift_forward')
        volumes_raw = volumes_raw.tz_convert(target_timezone)

        # Reindex to our full business day index and forward fill prices, fillna(0) volumes
        prices_lookback = prices_raw.reindex(fetch_dates_index).ffill()
        volumes_lookback = volumes_raw.reindex(fetch_dates_index).fillna(0)
        print(f"Asset price/volume data processed. Shape: {prices_lookback.shape}")

        # --- Crucial: Update asset list to only include those successfully downloaded ---
        original_asset_count = len(assets)
        assets = sorted(valid_assets) # Update the global assets list
        if len(assets) < original_asset_count:
            print(f"INFO: Asset list updated to {len(assets)} tickers with valid data.")
        if not assets: # Check if asset list became empty
             print("CRITICAL ERROR: No assets remaining after data download/validation. Exiting.")
             exit()
        # -------------------------------------------------------------------------------

    else:
        print("ERROR: No valid asset price data could be fetched or processed. Exiting.")
        exit() # Exit if no asset data

    # Process Benchmark Data
    if not data_benchmark.empty:
        if not isinstance(data_benchmark.index, pd.DatetimeIndex): data_benchmark.index = pd.to_datetime(data_benchmark.index)
        adj_close_key_bm = 'Adj Close' if 'Adj Close' in data_benchmark.columns else 'Close'
        if adj_close_key_bm not in data_benchmark.columns:
             print(f"ERROR: Benchmark price column ('{adj_close_key_bm}') not found.")
             # Create empty series as fallback
             benchmark_prices_lookback = pd.Series(dtype=float, index=fetch_dates_index, name=benchmark_ticker)
        else:
            benchmark_prices_raw = data_benchmark[adj_close_key_bm]
            if benchmark_prices_raw.index.tz is None: benchmark_prices_raw.index = benchmark_prices_raw.index.tz_localize('UTC', ambiguous='infer', nonexistent='shift_forward')
            benchmark_prices_raw = benchmark_prices_raw.tz_convert(target_timezone)
            benchmark_prices_lookback = benchmark_prices_raw.reindex(fetch_dates_index).ffill()
            print(f"Benchmark data processed. Length: {len(benchmark_prices_lookback)}")
    else:
        print("ERROR: Benchmark data could not be fetched.")
        # Create empty series if benchmark fetch failed
        benchmark_prices_lookback = pd.Series(dtype=float, index=fetch_dates_index, name=benchmark_ticker)

except Exception as e:
    print(f"\nERROR during data download: {e}"); traceback.print_exc()
    print("CRITICAL ERROR: Data download failed. Exiting.")
    exit()


# Create dataframes for the analysis period by slicing lookback data
# Ensure slicing uses the potentially updated 'assets' list
prices = prices_lookback.loc[dates_index, assets].copy()
volumes = volumes_lookback.loc[dates_index, assets].copy()
benchmark_prices = benchmark_prices_lookback.loc[dates_index].copy()

# --- Robust check for empty or all-NaN core data ---
prices_all_nan = False
if not prices.empty:
    prices_all_nan = prices.isna().all().all() # Check if ALL values are NaN

benchmark_all_nan = False
if not benchmark_prices.empty and isinstance(benchmark_prices, pd.Series): # Ensure it's a Series
    benchmark_all_nan = benchmark_prices.isna().all() # Check if ALL values are NaN

if prices.empty or benchmark_prices.empty or prices_all_nan or benchmark_all_nan:
     print("CRITICAL ERROR: Prices or Benchmark data is invalid (empty or all NaN) for the analysis period. Exiting.")
     exit()
# --- End of robust check ---


# --- Fetch Industry Data ---
print("\n--- Fetching Industry Classification Data ---")
def fetch_industry_data(tickers):
    industry_dict = {}
    missing_industries = []
    for ticker_str in tqdm(tickers, desc="Fetching Industries"):
        try:
            ticker_obj = yf.Ticker(ticker_str)
            # info_data = ticker_obj.fast_info # Potentially faster, fewer fields
            info_data = ticker_obj.info # Slower but more comprehensive
            industry = info_data.get('industry', 'Unknown')
            sector = info_data.get('sector', 'Unknown') # Get sector too
            # Prefer industry, fall back to sector, then Unknown
            if industry in [None, '', 'N/A', 'Unknown']:
                 industry = sector if sector not in [None, '', 'N/A', 'Unknown'] else 'Unknown'
            final_industry = industry if industry is not None else 'Unknown' # Ensure value is not None

            if final_industry == 'Unknown': missing_industries.append(ticker_str)
            industry_dict[ticker_str] = final_industry
            time.sleep(0.05) # Small delay to avoid potential rate limiting
        except Exception as e_ind:
            print(f"WARN: Error fetching industry for {ticker_str}: {e_ind}") # Show specific error
            industry_dict[ticker_str] = 'Unknown'
            missing_industries.append(ticker_str)

    if missing_industries: print(f"WARNING: Could not reliably fetch industry/sector for: {list(set(missing_industries))}")
    return pd.Series(industry_dict, name='industry')

if assets: # Only fetch if we have assets
    asset_industries = fetch_industry_data(assets)
    industry_dummies_static = pd.DataFrame()
    if not asset_industries.empty:
        # Create dummies, ensuring they align with the final 'assets' list
        industry_dummies_static = pd.get_dummies(asset_industries.reindex(assets).fillna('Unknown'), dummy_na=False, prefix='Ind').astype(int)
        # Drop 'Ind_Unknown' if other industries exist and it's all zero, or if only Unknown exists keep it.
        if 'Ind_Unknown' in industry_dummies_static.columns and len(industry_dummies_static.columns) > 1:
             if not industry_dummies_static['Ind_Unknown'].any():
                  industry_dummies_static = industry_dummies_static.drop('Ind_Unknown', axis=1)
        industry_dummies_static.index.name = 'asset'
        print(f"Created Static Industry Dummies shape: {industry_dummies_static.shape}")
        if industry_dummies_static.empty:
             print("WARN: Industry dummies became empty after processing (e.g., only 'Unknown' dropped). Creating default.")
             industry_dummies_static = pd.DataFrame({'Ind_NoIndustry': 1}, index=assets).astype(int)
    else:
        print("WARNING: Could not create industry dummies (fetch returned empty). Creating default.")
        industry_dummies_static = pd.DataFrame({'Ind_NoIndustry': 1}, index=assets).astype(int)
        industry_dummies_static.index.name = 'asset'
else:
    print("WARNING: No assets defined, skipping industry fetch.")
    asset_industries = pd.Series(dtype=str, name='industry')
    industry_dummies_static = pd.DataFrame(index=pd.Index([], name='asset')) # Ensure empty df has index


# ================================================================
# === LOAD OR DEFINE YOUR PRE-CALCULATED FACTORS HERE ===
# ================================================================
print("\n--- Loading/Defining Pre-calculated Factors ---")

# --- INPUT REQUIRED ---
# Option 1: Load from file (RECOMMENDED)
LOAD_FROM_FILE = True # SET TO TRUE TO LOAD FROM FILE
factors_file_path = "processed_alpha_data_values_1.csv" # OR .csv, .pkl etc.
# Expected format: See FORMAT 1 or FORMAT 2 descriptions below.

# Option 2: Define programmatically (like the dummy example)
CREATE_DUMMY_FACTORS = True # Set to False if loading from file

factors_input_df = pd.DataFrame() # Initialize

if LOAD_FROM_FILE:
    print(f"Attempting to load factors from: {factors_file_path}")
    if not os.path.exists(factors_file_path):
         print(f"ERROR: Factors file not found at {factors_file_path}")
    else:
        try:
            # Example loading parquet (adjust based on your file type)
            if factors_file_path.endswith(".parquet"):
                factors_input_df = pd.read_parquet(factors_file_path)
            elif factors_file_path.endswith(".csv"):
                # Adjust read_csv parameters as needed (e.g., index_col, parse_dates)
                # Assuming format 2 (stacked) for CSV example:
                factors_input_df = pd.read_csv(factors_file_path, index_col=[0, 1], parse_dates=[0])
                # Set index names if not read automatically
                if factors_input_df.index.names != ['date', 'asset']:
                     print("WARN: Setting loaded CSV index names to ['date', 'asset'].")
                     factors_input_df.index.names = ['date', 'asset']
            elif factors_file_path.endswith(".pkl"):
                factors_input_df = pd.read_pickle(factors_file_path)
            else:
                print(f"ERROR: Unsupported file format: {factors_file_path}")

            if not factors_input_df.empty:
                print(f"Successfully loaded factors from file. Initial shape: {factors_input_df.shape}")

                # --- Post-load processing based on format ---
                # Check if loaded data is Format 2 (stacked: Index=(date, asset), Columns=FactorNames)
                if isinstance(factors_input_df.index, pd.MultiIndex) and list(factors_input_df.index.names) == ['date', 'asset']:
                    print("INFO: Loaded data appears to be in Format 2 (Stacked). Unstacking...")
                    try:
                        factors_input_df_wide = factors_input_df.unstack(level='asset')
                        factors_input_df_wide.columns = pd.MultiIndex.from_tuples(
                            [(col_name, asset_name) for col_name, asset_name in factors_input_df_wide.columns],
                            names=['factor_name', 'asset']
                        )
                        factors_input_df = factors_input_df_wide # Overwrite with Format 1
                        print("Successfully unstacked factors to Format 1 (Wide).")
                    except Exception as e_unstack_load:
                        print(f"ERROR: Could not unstack the loaded factor DataFrame: {e_unstack_load}")
                        factors_input_df = pd.DataFrame() # Invalidate on error
                # Assume loaded data is already Format 1 (wide: Index=date, Columns=(factor_name, asset))
                elif isinstance(factors_input_df.index, pd.DatetimeIndex) and isinstance(factors_input_df.columns, pd.MultiIndex):
                    print("INFO: Loaded data appears to be in Format 1 (Wide).")
                    # Ensure column level names are correct
                    if list(factors_input_df.columns.names) != ['factor_name', 'asset']:
                        print("WARN: Renaming columns to ['factor_name', 'asset']. Please verify.")
                        factors_input_df.columns.names = ['factor_name', 'asset']
                else:
                    print("ERROR: Loaded DataFrame format is not recognized as Format 1 or Format 2.")
                    factors_input_df = pd.DataFrame() # Invalidate

        except Exception as e_load:
            print(f"ERROR: Failed to load or process factors file: {e_load}")
            traceback.print_exc()
            factors_input_df = pd.DataFrame()

elif CREATE_DUMMY_FACTORS:
    # Create dummy stacked data for demonstration:
    print("INFO: Creating dummy factor data for demonstration...")
    if not dates_index.empty and assets:
        multi_idx = pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset'])
        dummy_data = {
            'Factor_Dummy_1': np.random.randn(len(multi_idx)),
            'Factor_Dummy_2': np.random.rand(len(multi_idx)) - 0.5
        }
        factors_input_df_stacked = pd.DataFrame(dummy_data, index=multi_idx)
        print(f"Dummy stacked factors created. Shape: {factors_input_df_stacked.shape}")

        # --- Convert FORMAT 2 (Stacked) to FORMAT 1 (Wide - Preferred by the script) ---
        try:
            factors_input_df = factors_input_df_stacked.unstack(level='asset')
            factors_input_df.columns = pd.MultiIndex.from_tuples(
                [(col_name, asset_name) for col_name, asset_name in factors_input_df.columns],
                names=['factor_name', 'asset']
            )
            print("Successfully unstacked dummy factors to Format 1 (Wide).")
        except Exception as e_unstack_dummy:
            print(f"ERROR: Could not unstack the dummy factor DataFrame: {e_unstack_dummy}")
            factors_input_df = pd.DataFrame() # Assign empty df on error
    else:
        print("ERROR: Cannot create dummy factors - dates_index or assets are empty.")
        factors_input_df = pd.DataFrame()
else:
    print("INFO: No factor loading or creation specified.")


# --- Validation and Final Preparation ---
precalculated_factors_df = pd.DataFrame() # Initialize final df

if not factors_input_df.empty:
    # Ensure index is DatetimeIndex and has correct timezone
    if not isinstance(factors_input_df.index, pd.DatetimeIndex):
        try:
            factors_input_df.index = pd.to_datetime(factors_input_df.index)
        except Exception as e_conv:
            print(f"ERROR: Could not convert factor index to DatetimeIndex: {e_conv}. Invalidating factors.")
            factors_input_df = pd.DataFrame()

    if not factors_input_df.empty: # Check again after potential invalidation
        if factors_input_df.index.tz is None:
            try:
                print("INFO: Localizing factor index timezone...")
                factors_input_df.index = factors_input_df.index.tz_localize(target_timezone, ambiguous='infer', nonexistent='shift_forward')
            except TypeError: # Already localized
                 pass
            except Exception as e_tz:
                print(f"ERROR: Could not localize factor index timezone: {e_tz}. Invalidating factors.")
                factors_input_df = pd.DataFrame()

        if not factors_input_df.empty and factors_input_df.index.tz != target_timezone:
            try:
                print("INFO: Converting factor index timezone...")
                factors_input_df.index = factors_input_df.index.tz_convert(target_timezone)
            except Exception as e_tz_conv:
                print(f"ERROR: Could not convert factor index timezone: {e_tz_conv}. Invalidating factors.")
                factors_input_df = pd.DataFrame()

    # Ensure columns are MultiIndex ['factor_name', 'asset']
    if not factors_input_df.empty:
        if isinstance(factors_input_df.columns, pd.MultiIndex) and list(factors_input_df.columns.names) == ['factor_name', 'asset']:
             # Reindex to ensure all factors/assets/dates are present
             factor_names_present = factors_input_df.columns.get_level_values('factor_name').unique()
             # Ensure assets used for reindexing are the ones we have price data for
             target_multi_columns = pd.MultiIndex.from_product([factor_names_present, assets], names=['factor_name', 'asset'])

             print(f"Reindexing loaded factors to match analysis dates ({len(dates_index)}) and assets ({len(assets)})...")
             # Reindex BOTH index and columns to match the analysis scope
             precalculated_factors_df = factors_input_df.reindex(index=dates_index, columns=target_multi_columns)
             # Check for excessive NaNs after reindexing
             nan_frac = precalculated_factors_df.isna().mean().mean() if not precalculated_factors_df.empty else 1.0
             if precalculated_factors_df.isna().all().all():
                  print("CRITICAL WARN: Factor DataFrame is ALL NaNs after reindexing. Check date/asset alignment. Analysis will likely fail.")
             elif nan_frac > 0.9: # Example threshold
                  print(f"WARN: Factor DataFrame has >90% NaNs ({nan_frac:.1%}) after reindexing.")

             print(f"Final precalculated factors DataFrame ready. Shape: {precalculated_factors_df.shape}")
             available_factors = precalculated_factors_df.columns.get_level_values('factor_name').unique().tolist()
             print(f"Available factors: {available_factors}")
             if not available_factors:
                  print("ERROR: No factor names found after processing. Invalidating.")
                  precalculated_factors_df = pd.DataFrame()

        else:
             print("ERROR: Processed factor DataFrame columns are not MultiIndex named ['factor_name', 'asset']. Invalidating factors.")
             precalculated_factors_df = pd.DataFrame()

else:
    print("ERROR: No factor input data loaded or created.")
    precalculated_factors_df = pd.DataFrame()

# Final check before analysis loop
if precalculated_factors_df.empty:
     print("\nCRITICAL ERROR: The precalculated_factors_df is empty or invalid after loading/processing. Cannot proceed with analysis.")
     exit()
# ================================================================
# === END OF FACTOR LOADING SECTION ===
# ================================================================


# --- Calculate Style Factors (Do ONCE before loop) ---
print("\n--- Calculating Style Factors (Beta, Size Proxy, Liquidity Proxy, Residual Volatility) ---")
# Initialize with correct index names BUT NO DATA YET
style_factors = pd.DataFrame(index=pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset'])) # Base structure
style_factors_calculated = {} # Store components temporarily

min_periods_neut = max(10, neutralization_lookback // 2)
try:
    # Use lookback dataframes here
    print(f"Calculating style factors using {neutralization_lookback}-day lookback...")

    # 1. Size/Liquidity Proxy: Log of rolling average dollar volume
    if not prices_lookback.empty and not volumes_lookback.empty:
        dollar_volume_lb = prices_lookback.loc[:, assets] * volumes_lookback.loc[:, assets] # Ensure asset alignment
        rolling_dollar_vol_lb = dollar_volume_lb.rolling(neutralization_lookback, min_periods=min_periods_neut).mean()
        epsilon = 1e-9 # Smaller epsilon
        size_liquidity_proxy_df = np.log1p(rolling_dollar_vol_lb + epsilon)
        size_liquidity_proxy_df = size_liquidity_proxy_df.replace([np.inf, -np.inf], np.nan)
        # Slice to analysis dates *before* stacking
        size_liq_proxy_stacked = size_liquidity_proxy_df.loc[dates_index, assets].stack(future_stack=True).rename('size_liquidity_proxy')
        size_liq_proxy_stacked.index.names = ['date', 'asset'] # Set Index Names
        if not size_liq_proxy_stacked.dropna().empty: # Check if not all NaN
            style_factors_calculated['size_liquidity_proxy'] = size_liq_proxy_stacked
            print(" - Calculated Size/Liquidity Proxy.")
        else: print("WARN: Size/Liquidity Proxy resulted in empty or all-NaN series.")
    else: print("WARN: Skipping Size/Liquidity Proxy calc due to missing price/volume lookback data.")


    # 2. Beta: Rolling regression against benchmark
    if not prices_lookback.empty and not benchmark_prices_lookback.dropna().empty: # Check benchmark has data
        asset_returns_lb = prices_lookback.loc[:, assets].pct_change() # Ensure asset alignment
        benchmark_returns_lb = benchmark_prices_lookback.pct_change()

        # Ensure benchmark returns are not all NaN before proceeding
        if benchmark_returns_lb.dropna().empty:
             print("WARN: Benchmark returns are all NaN in lookback period. Skipping Beta calculation.")
        else:
            aligned_benchmark_ret_lb = benchmark_returns_lb.reindex(asset_returns_lb.index).ffill()
            X_beta_base = sm.add_constant(aligned_benchmark_ret_lb.dropna()) # Prepare RHS once
            betas = {} # Re-initialize dict for beta results specifically

            print(" - Calculating Beta (this may take a while)...")
            with tqdm(total=len(assets), desc="Calculating Beta", leave=False) as pbar:
                for asset in assets:
                    y_beta = asset_returns_lb[asset].dropna()

                    # --- Robust Beta Calculation Start ---
                    if y_beta.empty or X_beta_base.empty:
                        betas[asset] = pd.Series(np.nan, index=dates_index, name=asset) # Assign NaN series aligned with main index
                        pbar.update(1)
                        continue # Skip to next asset

                    common_idx_beta = X_beta_base.index.intersection(y_beta.index)

                    if len(common_idx_beta) >= neutralization_lookback: # Use >= lookback for min_nobs logic
                        X_beta_aligned = X_beta_base.loc[common_idx_beta]
                        y_beta_aligned = y_beta.loc[common_idx_beta]

                        if y_beta_aligned.empty or X_beta_aligned.empty:
                             betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                             pbar.update(1)
                             continue

                        try:
                            # Use imported RollingOLS directly
                            rols = RollingOLS(endog=y_beta_aligned, exog=X_beta_aligned,
                                             window=neutralization_lookback, min_nobs=min_periods_neut)
                            results = rols.fit()
                            # Check if params DataFrame is not empty and has enough columns
                            if not results.params.empty and results.params.shape[1] > 1:
                                 beta_series = results.params.iloc[:, 1] # Beta coeff index 1
                                 # Reindex to target dates_index AFTER calculation for this asset
                                 betas[asset] = beta_series.reindex(dates_index).ffill().bfill()
                            else:
                                 #print(f"WARN [{asset}]: RollingOLS params empty or misshaped.")
                                 betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)

                        except IndexError: # Catch index error if params structure unexpected
                            #print(f"WARN [{asset}]: RollingOLS IndexError (likely bad fit).")
                            betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                        except MissingDataError: # Catch if not enough observations for a window
                            #print(f"WARN [{asset}]: RollingOLS MissingDataError.")
                            betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                        except LinAlgError: # Catch linear algebra errors (e.g., singular matrix)
                            #print(f"WARN [{asset}]: RollingOLS LinAlgError.")
                            betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                        except ValueError as e_ols_val: # Catch potential value errors during fit
                            #print(f"WARN [{asset}]: RollingOLS ValueError: {e_ols_val}")
                            betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                        except Exception as e_beta_sm: # Catch other unexpected errors
                            #print(f"WARN [{asset}]: RollingOLS failed unexpectedly: {e_beta_sm}")
                            betas[asset] = pd.Series(np.nan, index=dates_index, name=asset)
                    else:
                        # Not enough common data points for reliable rolling beta
                        betas[asset] = pd.Series(np.nan, index=dates_index, name=asset) # Assign NaN series
                    pbar.update(1)
                    # --- Robust Beta Calculation End ---

            # --- Concatenate Beta results ---
            if betas: # Check if the betas dictionary is not empty
                try:
                    # Filter out any potential non-Series items just in case
                    valid_betas = {k: v for k, v in betas.items() if isinstance(v, pd.Series)}
                    if valid_betas:
                        beta_df = pd.concat(valid_betas.values(), axis=1, keys=valid_betas.keys()) # Use values and keys
                        beta_df.columns.name = 'asset' # Name the column index
                        # Stack the dataframe (already indexed by dates_index)
                        beta_stacked = beta_df.stack(future_stack=True).rename('beta')
                        beta_stacked.index.names = ['date', 'asset'] # Set Index Names
                        if not beta_stacked.dropna().empty: # Check if not all NaN
                            style_factors_calculated['beta'] = beta_stacked
                            print(" - Calculated Beta.")
                        else: print("WARN: Beta calculation resulted in empty or all-NaN series after stacking.")
                    else: print("WARN: No valid beta Series were generated.")
                except ValueError as e_concat_beta:
                     print(f"ERROR concatenating beta results: {e_concat_beta}")
                     print("WARN: Skipping Beta factor due to concatenation error.")
            else:
                print("WARN: No beta values could be calculated for any asset.")
    else: print("WARN: Skipping Beta calc due to missing price/benchmark lookback data.")


    # 3. Residual Volatility: Rolling std dev of returns
    if not prices_lookback.empty:
        if 'asset_returns_lb' not in locals(): # Calculate if not done for beta
             asset_returns_lb = prices_lookback.loc[:, assets].pct_change() # Ensure asset alignment
        rolling_std_ret = asset_returns_lb.rolling(neutralization_lookback, min_periods=min_periods_neut).std()
        # Slice to analysis dates *before* stacking
        res_vol_stacked = rolling_std_ret.loc[dates_index, assets].stack(future_stack=True).rename('residual_vol')
        res_vol_stacked.index.names = ['date', 'asset'] # Set Index Names
        if not res_vol_stacked.dropna().empty: # Check if not all NaN
             style_factors_calculated['residual_vol'] = res_vol_stacked
             print(" - Calculated Residual Volatility.")
        else: print("WARN: Residual Volatility calculation resulted in empty or all-NaN series.")
    else: print("WARN: Skipping Residual Volatility calc due to missing price lookback data.")


    # --- Combine all calculated factors at the end ---
    if style_factors_calculated:
         # Ensure all components are Series before concat
         valid_components = {k: v for k, v in style_factors_calculated.items() if isinstance(v, pd.Series)}
         if valid_components:
             style_factors = pd.concat(valid_components.values(), axis=1) # Combine valid Series into DF
             # Reindex just in case some date/asset combos were missing in all factors
             style_factors = style_factors.reindex(pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset']))
             print(f"Style factors calculation finished. Final Shape: {style_factors.shape}")
         else:
              print("WARN: No valid style factor components were calculated.")
              style_factors = pd.DataFrame(index=pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset']))
    else:
         print("WARN: No style factors were successfully calculated.")
         style_factors = pd.DataFrame(index=pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset']))


except Exception as e_style:
    print(f"ERROR calculating style factors: {e_style}")
    traceback.print_exc()
    style_factors = pd.DataFrame(index=pd.MultiIndex.from_product([dates_index, assets], names=['date', 'asset']))
    print("WARN: Style factors calculation failed. Proceeding without them for neutralization.")


# --- Analysis Function Definitions ---
print("\n--- Defining/Importing Analysis Functions ---")

def calculate_forward_returns(prices_df, periods):
    """Calculates forward returns for multiple periods. Corrected version 3."""
    # prices_df: Index=date, Columns=assets
    all_fwd_returns = {} # Store DataFrames for each period

    if prices_df.empty:
         print("ERROR [Fwd Ret]: Input prices_df is empty.")
         return pd.DataFrame(index=pd.MultiIndex([[],[]], [[],[]], names=['date','asset']), columns=analysis_periods_str)

    for p in periods:
        fwd_ret_col_name = f'{p}D_fwd_ret'
        # Calculate returns for all assets for this period 'p'
        shifted_price = prices_df.shift(-p)
        # Ensure alignment before division, handle potential NaNs gracefully
        returns_p = (shifted_price / prices_df - 1).replace([np.inf, -np.inf], np.nan)
        all_fwd_returns[fwd_ret_col_name] = returns_p

    if not all_fwd_returns:
        print("WARN [Fwd Ret]: No forward returns calculated.")
        return pd.DataFrame(index=pd.MultiIndex([[],[]], [[],[]], names=['date','asset']), columns=analysis_periods_str)

    # Concat creates MultiIndex columns: ('1D_fwd_ret', 'asset1'), ('3D_fwd_ret', 'asset1'), ...
    combined_fwd_returns_wide = pd.concat(all_fwd_returns, axis=1)
    combined_fwd_returns_wide.columns.names = ['period', 'asset'] # Name the column levels

    # Stack the 'asset' level from columns to index to get format:
    # Index = MultiIndex('date', 'asset'), Columns = Index(['1D_fwd_ret', '3D_fwd_ret', ...])
    fwd_returns_stacked = combined_fwd_returns_wide.stack(level='asset', future_stack=True)
    fwd_returns_stacked.index.names = ['date', 'asset'] # Ensure final index names are correct
    # Ensure columns are named correctly (should be the periods after stacking 'asset')
    fwd_returns_stacked.columns.name = 'period' # Name the columns index

    return fwd_returns_stacked

def get_quantile_assignments(factor_df, num_quantiles=5):
    """Assigns assets to quantiles based on factor values for each date."""
    # Input factor_df: Index=(date, asset), Column='factor'
    if factor_df.empty: return pd.DataFrame(columns=['quantile'], index=factor_df.index) # Handle empty input

    factor_col_name = 'factor'
    if factor_col_name not in factor_df.columns:
        if isinstance(factor_df, pd.Series) and factor_df.name == factor_col_name:
             factor_df = factor_df.to_frame()
        elif not factor_df.empty: # Try using the first column if name isn't 'factor'
             original_col = factor_df.columns[0]
             factor_df = factor_df[[original_col]].rename(columns={original_col: factor_col_name})
        else: # Cannot proceed if empty and no factor column
            return pd.DataFrame(columns=['quantile'], index=factor_df.index)

    # Use transform to handle broadcasting within groups safely
    quantiles = factor_df.groupby(level='date')[factor_col_name].transform(
        lambda x: pd.qcut(x, num_quantiles, labels=False, duplicates='drop')
    ) + 1 # Labels 1 to N
    quantiles = quantiles.rename('quantile')

    return quantiles.to_frame() # Return as DataFrame


def quantile_analysis(analysis_data, factor_display_name, num_quantiles=5, ret_col='1D_fwd_ret'):
    """Performs quantile return analysis."""
    # analysis_data: Index=(date, asset), Columns=['factor', ret_col]
    if ret_col not in analysis_data.columns:
        print(f"WARN [Quantile Analysis]: Return column '{ret_col}' not found.")
        return None, None
    if 'factor' not in analysis_data.columns:
        print(f"WARN [Quantile Analysis]: Factor column 'factor' not found.")
        return None, None
    if analysis_data.empty or analysis_data[['factor', ret_col]].isna().all().all():
         print(f"WARN [Quantile Analysis]: Input data empty or all NaN for {factor_display_name}/{ret_col}.")
         return None, None

    quantile_assignments = get_quantile_assignments(analysis_data[['factor']], num_quantiles)
    if quantile_assignments.empty or quantile_assignments['quantile'].isna().all():
         print(f"WARN [Quantile Analysis]: Could not assign quantiles for {factor_display_name}.")
         return None, None

    data_with_quantiles = analysis_data.join(quantile_assignments, how='inner').dropna(subset=['quantile'])
    if data_with_quantiles.empty: # Check after join/dropna
         print(f"WARN [Quantile Analysis]: Data empty after joining quantiles for {factor_display_name}.")
         return None, None

    # Mean return per quantile (averaged over time)
    mean_ret_by_quantile = data_with_quantiles.groupby('quantile')[ret_col].mean()

    # Cumulative return per quantile
    daily_mean_ret_by_q = data_with_quantiles.groupby(['date', 'quantile'])[ret_col].mean().unstack(level='quantile')
    # Fill missing daily quantile returns (e.g., if a quantile had no members) with 0 for cumulative calc
    daily_mean_ret_by_q = daily_mean_ret_by_q.fillna(0)
    # Calculate geometric cumulative returns
    cumulative_ret_by_q = (1 + daily_mean_ret_by_q).cumprod() - 1

    return mean_ret_by_quantile, cumulative_ret_by_q


def calculate_quantile_turnover(quantile_assignments, num_quantiles=5):
    """Calculates quantile turnover."""
    # quantile_assignments: Index=(date, asset), Column='quantile'
    if quantile_assignments.empty or quantile_assignments['quantile'].isna().all():
        print("WARN [Turnover]: Input quantile assignments are empty or all NaN.")
        return pd.DataFrame() # Return empty df

    turnover_results = {}
    quantiles_unstacked = quantile_assignments['quantile'].unstack(level='asset')

    # Ensure index is sorted for shift to work correctly
    quantiles_unstacked = quantiles_unstacked.sort_index()

    for q in range(1, num_quantiles + 1):
        quantile_members = (quantiles_unstacked == q)
        prev_members = quantile_members.shift(1)

        # Align and stack, keeping only days where both current and previous exist
        combined = pd.concat(
            [quantile_members.stack(future_stack=True).rename('current'),
             prev_members.stack(future_stack=True).rename('previous')],
            axis=1
        ).dropna() # Drop rows where either is NaN (i.e., first day, or if assets change)

        if combined.empty:
             # Handle case with only one day of data or no overlap
             daily_turnover_series = pd.Series(np.nan, index=quantiles_unstacked.index)
        else:
            def daily_turnover(group):
                # Check if group is empty or has wrong structure
                if group.empty or not all(c in group.columns for c in ['current', 'previous']):
                    return np.nan

                stayed = (group['current'] & group['previous']).sum()
                entered = (group['current'] & ~group['previous']).sum()
                exited = (~group['current'] & group['previous']).sum()
                total_current = group['current'].sum()
                total_previous = group['previous'].sum()

                avg_size = (total_current + total_previous) / 2.0
                if avg_size < 1e-6: return 0.0 # Handle near-zero avg size

                # Using: max(entered, exited) / avg_size
                traded = max(entered, exited)
                return traded / avg_size if avg_size > 0 else 0.0

            # Apply daily turnover calculation
            daily_turnover_series = combined.groupby(level='date').apply(daily_turnover)
            # Reindex to original dates index to include days with NaN turnover
            daily_turnover_series = daily_turnover_series.reindex(quantiles_unstacked.index)


        turnover_results[f'Q{q}_Turnover'] = daily_turnover_series

    turnover_df = pd.DataFrame(turnover_results)
    if not turnover_df.empty:
        turnover_df['Mean_Turnover'] = turnover_df.mean(axis=1)
    return turnover_df


def calculate_ic(analysis_data, factor_display_name, ret_col='1D_fwd_ret', method='spearman'):
    """Calculates Information Coefficient (IC)."""
    # analysis_data: Index=(date, asset), Columns=['factor', ret_col]
    if ret_col not in analysis_data.columns or 'factor' not in analysis_data.columns:
         print(f"WARN [IC]: Missing 'factor' or '{ret_col}' for {factor_display_name}")
         return None, None
    if analysis_data.empty or analysis_data[['factor', ret_col]].isna().all().all():
         print(f"WARN [IC]: Input data empty or all NaN for {factor_display_name}/{ret_col}.")
         return None, None

    def ic_calc(group):
        group_clean = group[['factor', ret_col]].dropna()
        if len(group_clean) < 3: return np.nan # Need >= 3 points for reliable correlation? Usually 2 is min.
        try:
            # Check for zero variance before calculating correlation
            factor_std_dev = group_clean['factor'].std()
            ret_std_dev = group_clean[ret_col].std()
            if pd.isna(factor_std_dev) or factor_std_dev < 1e-9 or pd.isna(ret_std_dev) or ret_std_dev < 1e-9:
                 return 0.0 # Treat constant series as zero correlation

            if method == 'spearman':
                coeff, p_val = spearmanr(group_clean['factor'], group_clean[ret_col])
                return coeff if pd.notna(coeff) else 0.0 # Return 0 if spearman returns NaN
            elif method == 'pearson':
                coeff = group_clean['factor'].corr(group_clean[ret_col], method='pearson')
                return coeff if pd.notna(coeff) else 0.0 # Return 0 if pearson returns NaN
            else: return np.nan
        except ValueError: # Handle other potential errors (e.g., from spearmanr)
             return np.nan

    daily_ic = analysis_data.groupby(level='date').apply(ic_calc)
    daily_ic.name = f'IC_{method}' # Rename the resulting Series

    # Summarize IC
    ic_mean = daily_ic.mean()
    ic_std = daily_ic.std()
    icir = ic_mean / ic_std if pd.notna(ic_std) and ic_std > 1e-9 else np.nan # Avoid div by zero/tiny std
    hit_rate = (daily_ic > 1e-9).mean() if not daily_ic.dropna().empty else np.nan # Use > small epsilon for hit rate
    obs_days = daily_ic.count() # Count non-NaN IC days

    ic_summary = pd.Series({
        'Mean IC': ic_mean,
        'Std Dev IC': ic_std,
        'ICIR': icir,
        'Hit Rate (>0)': hit_rate,
        'Observations (Days)': obs_days
    }, name=ret_col) # Use ret_col as the Series name

    return ic_summary, daily_ic.to_frame() # Return daily IC as DataFrame


def calculate_factor_returns(analysis_data, factor_display_name, ret_col='1D_fwd_ret'):
    """Calculates factor returns (e.g., long/short portfolio based on factor)."""
    # analysis_data: Index=(date, asset), Columns=['factor', ret_col]
    if ret_col not in analysis_data.columns or 'factor' not in analysis_data.columns:
        print(f"WARN [Factor Returns]: Missing 'factor' or '{ret_col}' for {factor_display_name}")
        return None, None, None, None
    if analysis_data.empty or analysis_data[['factor', ret_col]].isna().all().all():
         print(f"WARN [Factor Returns]: Input data empty or all NaN for {factor_display_name}/{ret_col}.")
         return None, None, None, None

    # 1. Standardize Factor (cross-sectionally)
    factor_std = analysis_data.groupby(level='date')['factor'].transform(
        lambda x: (x - x.mean()) / x.std() if pd.notna(x.std()) and x.std() > 1e-9 else (x - x.mean()) # Handle zero/tiny std dev
    ).fillna(0) # Fill NaNs after standardization (e.g., single asset days) with 0 weight

    # 2. Calculate Weighted Return for each day
    analysis_data_temp = analysis_data[[ret_col]].copy() # Only need return col
    analysis_data_temp['factor_std'] = factor_std
    analysis_data_temp['weighted_ret'] = analysis_data_temp['factor_std'] * analysis_data_temp[ret_col]

    # --- Daily Factor Return: Weighted average return ---
    # Sum of (weight * return) / Sum of abs(weights) <-- For dollar neutral L/S
    sum_weighted_ret = analysis_data_temp.groupby(level='date')['weighted_ret'].sum()
    sum_abs_weights = analysis_data_temp.groupby(level='date')['factor_std'].apply(lambda x: x.abs().sum())
    # Avoid division by zero/NaN if sum of abs weights is zero/NaN for a day
    daily_factor_return = (sum_weighted_ret / sum_abs_weights.replace(0, np.nan)).dropna()
    daily_factor_return.name = 'factor_daily_ret'


    # 3. Calculate Cumulative Return
    cumulative_factor_return = pd.Series(index=daily_factor_return.index, dtype=float)
    if not daily_factor_return.empty:
        cumulative_factor_return = (1 + daily_factor_return).cumprod() - 1
    cumulative_factor_return.name = 'factor_cum_ret'

    # 4. Calculate Annualized Statistics
    ann_factor = 252 # Assuming 252 trading days per year
    num_days = len(daily_factor_return)
    ann_ret, ann_vol, sharpe = np.nan, np.nan, np.nan # Defaults
    if num_days > 5: # Require min days for meaningful stats
         mean_daily_ret = daily_factor_return.mean()
         std_daily_ret = daily_factor_return.std()
         if pd.notna(mean_daily_ret): ann_ret = mean_daily_ret * ann_factor
         if pd.notna(std_daily_ret) and std_daily_ret > 1e-9: # Avoid div by tiny std
              ann_vol = std_daily_ret * np.sqrt(ann_factor)
              if pd.notna(ann_ret) and ann_vol > 1e-9 : sharpe = ann_ret / ann_vol # Ensure vol > 0

    ann_stats = pd.Series({
        'Annualized Return': ann_ret,
        'Annualized Volatility': ann_vol,
        'Sharpe Ratio': sharpe,
        'Observations (Days)': num_days
    }, name=ret_col) # Use ret_col as the Series name

    return daily_factor_return.to_frame(), cumulative_factor_return.to_frame(), None, ann_stats # Placeholder for drawdown

# --- FIX IS HERE ---
def calculate_forward_returns_for_decay(prices_df, max_lag):
    """Calculates forward returns for multiple lags up to max_lag."""
    fwd_rets_dict = {}
    print(f"Calculating fwd returns for decay (1 to {max_lag} days)...")
    if prices_df.empty:
         print("WARN [Decay FwdRets]: Input prices_df is empty.")
         return fwd_rets_dict

    shifted_prices = {lag: prices_df.shift(-lag) for lag in range(1, max_lag + 1)}
    with tqdm(total=max_lag, desc="Fwd Returns Decay", leave=False) as pbar: # Set leave=False
        for lag in range(1, max_lag + 1):
            ret_col_name = f'{lag}D_fwd_ret'
            fwd_ret_lag = (shifted_prices[lag] / prices_df - 1).replace([np.inf, -np.inf], np.nan)
            # Stack to get (date, asset) index
            fwd_ret_stacked = fwd_ret_lag.stack(future_stack=True).rename(ret_col_name)
            # <<< FIX: Set index names >>>
            fwd_ret_stacked.index.names = ['date', 'asset']
            # <<< END FIX >>>
            if not fwd_ret_stacked.dropna().empty: # Check not all NaN
                fwd_rets_dict[lag] = fwd_ret_stacked.dropna() # Store cleaned series
            pbar.update(1)
    print(f"Finished calculating {len(fwd_rets_dict)} forward returns for decay.")
    return fwd_rets_dict # Dict: {lag: Series(Index=(date,asset), Value=ret)}
# --- END FIX ---

def calculate_ic_decay(factor_series_clean, fwd_returns_for_decay_dict, max_lag, method='spearman'):
    """Calculates IC decay over multiple forward return periods."""
    # factor_series_clean: Series, Index=(date, asset), Name='factor'
    ic_decay_values = {}
    print("Calculating IC Decay...")
    if factor_series_clean.empty or not fwd_returns_for_decay_dict:
         print("WARN [IC Decay]: Factor series empty or no fwd returns provided.")
         return pd.Series(dtype=float, name=f'Mean_IC_{method}_Decay').rename_axis('Lag (Days)')

    with tqdm(total=max_lag, desc="IC Decay", leave=False) as pbar: # Set leave=False
        for lag in range(1, max_lag + 1):
            result_ic = np.nan # Default
            if lag in fwd_returns_for_decay_dict:
                fwd_ret_lag = fwd_returns_for_decay_dict[lag]
                if not fwd_ret_lag.empty:
                    # Ensure both series are frames for merge (safer)
                    factor_frame = factor_series_clean.to_frame()
                    ret_frame = fwd_ret_lag.to_frame()
                    # <<< Check index names before merge for debugging >>>
                    # print(f"DEBUG IC Decay Lag {lag}: Factor index names: {factor_frame.index.names}, Ret index names: {ret_frame.index.names}")
                    # <<< End Debug >>>
                    try:
                        aligned_decay = pd.merge(factor_frame, ret_frame,
                                                 left_index=True, right_index=True, how='inner')
                        aligned_decay = aligned_decay.dropna() # Drop rows with NaNs in either column

                        if len(aligned_decay) > 2: # Need enough points
                            # Calculate mean daily IC for this lag
                            def ic_calc_decay(group):
                                if len(group) < 3: return np.nan
                                try:
                                     # Check variance again
                                    factor_std_dev = group['factor'].std()
                                    ret_std_dev = group[fwd_ret_lag.name].std()
                                    if pd.isna(factor_std_dev) or factor_std_dev < 1e-9 or pd.isna(ret_std_dev) or ret_std_dev < 1e-9:
                                         return 0.0 # Treat constant series as zero correlation

                                    if method == 'spearman':
                                        coeff, p_val = spearmanr(group['factor'], group[fwd_ret_lag.name])
                                        return coeff if pd.notna(coeff) else 0.0
                                    elif method == 'pearson':
                                        coeff = group['factor'].corr(group[fwd_ret_lag.name], method='pearson')
                                        return coeff if pd.notna(coeff) else 0.0
                                    else: return np.nan
                                except ValueError: return np.nan # Handle other errors (e.g. spearmanr issue)

                            daily_ic_lag = aligned_decay.groupby(level='date').apply(ic_calc_decay)
                            result_ic = daily_ic_lag.mean() # Store the mean IC for this lag
                    except ValueError as e_merge_decay: # Catch specific merge errors
                         print(f"ERROR [IC Decay Lag {lag}]: Merge failed - {e_merge_decay}. Skipping lag.")
                         result_ic = np.nan # Ensure NaN if merge fails
                    except Exception as e_decay_calc: # Catch other errors during calculation
                         print(f"ERROR [IC Decay Lag {lag}]: Calculation failed - {e_decay_calc}. Skipping lag.")
                         result_ic = np.nan

            ic_decay_values[lag] = result_ic
            pbar.update(1)

    ic_decay_series = pd.Series(ic_decay_values, name=f'Mean_IC_{method}_Decay')
    ic_decay_series.index.name = 'Lag (Days)'
    return ic_decay_series

# --- END OF ANALYSIS FUNCTION DEFINITIONS ---


# --- Helper function to save results to Excel ---
# Defined once before the loop starts
def save_to_excel_combined(df_to_save, base_sheet_name, factor_disp_name, writer_obj):
     """Saves a dataframe to a sheet in the combined Excel file, handling naming and timezones."""
     sheet_name_raw = f"{factor_disp_name}_{base_sheet_name}"
     if len(sheet_name_raw) > 31:
          max_len, len_base, len_underscore = 31, len(base_sheet_name), 1
          available_for_factor = max_len - len_base - len_underscore
          if available_for_factor < 3: sheet_name = sheet_name_raw[:max_len] # Min 3 chars for factor part
          else: sheet_name = f"{factor_disp_name[:available_for_factor]}_{base_sheet_name}"
          print(f"WARN: Sheet name '{sheet_name_raw}' > 31 chars. Truncated to '{sheet_name}'.")
     else: sheet_name = sheet_name_raw

     if df_to_save is not None and not df_to_save.empty:
          try:
               df_copy = df_to_save.copy()
               # Remove timezone info for Excel compatibility
               if isinstance(df_copy.index, pd.DatetimeIndex): df_copy.index = df_copy.index.tz_localize(None)
               if isinstance(df_copy.columns, pd.DatetimeIndex): df_copy.columns = df_copy.columns.tz_localize(None)
               if isinstance(df_copy.index, pd.MultiIndex):
                   new_levels = [lvl.tz_localize(None) if isinstance(lvl, pd.DatetimeIndex) else lvl for lvl in df_copy.index.levels]
                   df_copy.index = df_copy.index.set_levels(new_levels)
               if isinstance(df_copy.columns, pd.MultiIndex):
                   new_levels = [lvl.tz_localize(None) if isinstance(lvl, pd.DatetimeIndex) else lvl for lvl in df_copy.columns.levels]
                   df_copy.columns = df_copy.columns.set_levels(new_levels)

               df_copy.to_excel(writer_obj, sheet_name=sheet_name)
               # print(f"DEBUG: Saved sheet '{sheet_name}'") # Optional debug
               return True # Indicate sheet was saved
          except Exception as e_save: print(f"ERROR saving sheet '{sheet_name}': {e_save}")
     else: print(f"INFO: No data to save for sheet '{sheet_name}'.")
     return False # Indicate sheet was not saved


# --- Calculate Forward Returns for Analysis & IC Decay (Do ONCE before loop) ---
if prices.empty or prices.isna().all().all():
    print("ERROR: Price data is empty or all NaN. Cannot calculate forward returns. Exiting.")
    exit()

print(f"\nCalculating forward returns for analysis periods: {fwd_ret_periods_int} days...")
forward_returns_df_stacked = calculate_forward_returns(prices.copy(), periods=fwd_ret_periods_int)

if forward_returns_df_stacked.empty:
    print("ERROR: Main forward returns calculation failed or resulted in empty data. Exiting.")
    exit()
# Check if expected column names exist
expected_ret_cols_present = all(col in forward_returns_df_stacked.columns for col in analysis_periods_str)
if not expected_ret_cols_present:
     print(f"ERROR: Missing expected forward return columns in calculated df. Expected: {analysis_periods_str}, Found: {forward_returns_df_stacked.columns.tolist()}")
     # exit() # Exit or proceed carefully

print(f"\nCalculating forward returns for IC decay (up to {MAX_DECAY_LAG} days)...")
fwd_returns_for_decay_dict = calculate_forward_returns_for_decay(prices.copy(), MAX_DECAY_LAG)
if not fwd_returns_for_decay_dict:
    print("WARN: Could not calculate forward returns for IC Decay. Decay analysis will be skipped.")


# =======================================================
# === Starting Factor Analysis Loop ===
# =======================================================
print(f"\nPreparing single Excel output file: {combined_output_filename}\n")

if precalculated_factors_df.empty:
    print("ERROR: No pre-calculated factors found or loaded. Skipping analysis loop.")
else:
    # --- Start Excel Writer context ---
    overall_success = False # Flag to track if ANY sheet gets written
    try:
        with pd.ExcelWriter(combined_output_filename, engine='openpyxl') as writer:
            unique_factor_names = precalculated_factors_df.columns.get_level_values('factor_name').unique()
            print(f"Analyzing {len(unique_factor_names)} factors found in the input DataFrame...")

            # --- Loop through each factor ---
            for factor_name in unique_factor_names:
                print(f"\n\n{'='*20} Processing Factor: {factor_name} {'='*20}")
                factor_timer_start = time.time() # Timer for each factor
                sheets_saved_this_factor = 0 # Count sheets for this factor

                # --- Extract Raw Factor ---
                try:
                    raw_factor_df = precalculated_factors_df.xs(factor_name, level='factor_name', axis=1).copy()
                    raw_factor_df.columns.name = 'asset'
                    raw_factor_df.index.name = 'date'
                except KeyError:
                     print(f"ERROR: Could not extract factor '{factor_name}' using xs. Skipping.")
                     continue
                except Exception as e_extract:
                     print(f"ERROR: Unexpected error extracting factor '{factor_name}': {e_extract}. Skipping.")
                     continue

                if raw_factor_df.empty or raw_factor_df.isna().all().all():
                    print(f"WARN: Raw factor data for {factor_name} is empty or all NaNs after extraction. Skipping.")
                    continue

                # --- Factor Neutralization ---
                print("\n--- Performing Factor Neutralization ---")
                neut_timer_start = time.time()
                neutralized_factor_df = pd.DataFrame(index=dates_index, columns=assets) # Reinitialize
                neutralization_succeeded = False

                # Check if any neutralization variables exist and align them
                has_industry = False
                industry_dummies_aligned = pd.DataFrame()
                if 'industry_dummies_static' in locals() and not industry_dummies_static.empty:
                     # Align index (assets) with the current final asset list
                     industry_dummies_aligned = industry_dummies_static.reindex(assets).fillna(0)
                     has_industry = not industry_dummies_aligned.empty

                has_style = False
                style_factors_aligned = pd.DataFrame()
                if 'style_factors' in locals() and not style_factors.empty and not style_factors.isna().all().all():
                     # Align style factors (which have MultiIndex date,asset) with raw_factor_df dates
                     # And ensure assets match the final 'assets' list
                     style_factors_aligned = style_factors.reindex(index=raw_factor_df.index, level='date')
                     # Filter style factors to only include current assets
                     valid_style_assets = style_factors_aligned.index.get_level_values('asset').unique().intersection(assets)
                     if not valid_style_assets.empty:
                         style_factors_aligned = style_factors_aligned[style_factors_aligned.index.get_level_values('asset').isin(valid_style_assets)]
                         has_style = not style_factors_aligned.dropna(how='all').empty
                     else: has_style = False


                if not has_industry and not has_style:
                    print("INFO: No neutralization variables available. Using raw factor.")
                    neutralized_factor_df = raw_factor_df.copy()
                    neutralization_succeeded = False
                else:
                    print("Running neutralization regression day by day...")
                    neutralized_residuals_list = []
                    # Use index from raw factor that has *some* data for iteration
                    valid_dates_for_neut = raw_factor_df.dropna(how='all').index

                    with tqdm(total=len(valid_dates_for_neut), desc=f"Neutralizing {factor_name}", leave=False) as pbar:
                        for date in valid_dates_for_neut:
                            factor_today = raw_factor_df.loc[date].dropna()
                            if factor_today.empty:
                                neutralized_residuals_list.append(pd.Series(np.nan, index=assets, name=date))
                                pbar.update(1); continue

                            X_list = []
                            valid_assets_today = factor_today.index

                            # Industry
                            if has_industry:
                                industry_today = industry_dummies_aligned.reindex(valid_assets_today).dropna(axis=1, how='all').fillna(0)
                                # Drop industry dummies that are constant (e.g., all zero after reindex)
                                industry_today = industry_today.loc[:, industry_today.nunique() > 1]
                                if not industry_today.empty: X_list.append(industry_today)

                            # Style Factors
                            style_today_aligned_assets = pd.DataFrame() # Init empty
                            if has_style and date in style_factors_aligned.index.get_level_values('date'):
                                try:
                                    style_today = style_factors_aligned.loc[pd.IndexSlice[date, :], :] # Use IndexSlice for robustness
                                    if not style_today.empty:
                                         # If only one style factor, it might be a Series, convert to frame
                                         if isinstance(style_today, pd.Series): style_today = style_today.to_frame()

                                         # Reindex style factors for today's valid assets and fill NaNs (e.g., with mean)
                                         style_fill_value = style_today.mean() # Calculate mean before reindexing
                                         style_today_aligned_assets = style_today.reindex(valid_assets_today, level='asset').fillna(style_fill_value)
                                         # Drop style factors that are all NaN after reindexing/filling
                                         style_today_aligned_assets = style_today_aligned_assets.dropna(axis=1, how='all') # Drop empty columns
                                         if not style_today_aligned_assets.empty:
                                             # Remove constant columns (important!) before adding model constant
                                             non_const_cols = style_today_aligned_assets.loc[:, style_today_aligned_assets.nunique() > 1]
                                             if not non_const_cols.empty: X_list.append(non_const_cols)
                                except KeyError: pass # Date might not exist in aligned style factors
                                except Exception as e_style_align:
                                     print(f"WARN: Error aligning style factors for {date}: {e_style_align}")


                            if not X_list:
                                residuals_today = factor_today
                            else:
                                try:
                                    X_today = pd.concat(X_list, axis=1).astype(float) # Ensure float type
                                    # Align Y (factor) and X (exposures) on common assets
                                    common_assets = factor_today.index.intersection(X_today.index)
                                    if common_assets.empty: # Handle case where no assets overlap after considering exposures
                                         residuals_today = factor_today # Fallback to raw
                                    else:
                                        Y_aligned = factor_today.loc[common_assets].astype(float)
                                        X_aligned = X_today.loc[common_assets]

                                        # Drop rows/cols with all NaNs AFTER alignment (robustness)
                                        X_aligned = X_aligned.dropna(axis=1, how='all').dropna(axis=0, how='all')
                                        Y_aligned = Y_aligned.loc[X_aligned.index] # Re-align Y

                                        # Check for sufficient data points vs predictors
                                        if Y_aligned.empty or X_aligned.empty or len(Y_aligned) <= X_aligned.shape[1]:
                                            residuals_today = factor_today # Fallback
                                        else:
                                            X_w_const = sm.add_constant(X_aligned, has_constant='add')
                                            model = sm.OLS(Y_aligned, X_w_const, missing='drop')
                                            results = model.fit()
                                            residuals_today = results.resid.reindex(Y_aligned.index).fillna(0) # Fill NaNs from regression with 0? Or keep NaN?

                                except LinAlgError: # Handle cases like singular matrix
                                     residuals_today = factor_today
                                except ValueError as e_ols_val: # Handle dimension mismatches etc.
                                     residuals_today = factor_today
                                except Exception as e_ols:
                                     print(f"WARN: OLS failed unexpectedly for {factor_name} on {date}: {e_ols}")
                                     residuals_today = factor_today # Fallback to raw on error

                            # Reindex residuals to full asset list, filling missing ones with NaN
                            neutralized_residuals_list.append(residuals_today.reindex(assets).fillna(np.nan))
                            pbar.update(1)

                    # --- Combine daily neutralized results ---
                    if neutralized_residuals_list:
                         neutralized_factor_df_temp = pd.concat(neutralized_residuals_list, axis=1).T
                         neutralized_factor_df_temp.index.name = 'date'
                         # Reindex to ensure all analysis dates are present (fills missing dates with NaN)
                         neutralized_factor_df = neutralized_factor_df_temp.reindex(dates_index)
                         neutralization_succeeded = True
                         print(f"Neutralization completed for {factor_name}. ({(time.time() - neut_timer_start):.2f}s)")
                    else:
                         print(f"WARN: Neutralization yielded no results for {factor_name}. Using raw factor.")
                         neutralized_factor_df = raw_factor_df.copy()
                         neutralization_succeeded = False


                # --- Analysis Execution ---
                print("\n--- Starting Factor Analysis ---")
                analysis_timer_start = time.time()
                factor_to_analyze_df = None
                factor_source = "None"

                # Decide which factor version to use for analysis
                if neutralization_succeeded and not neutralized_factor_df.isna().all().all():
                    factor_to_analyze_df = neutralized_factor_df.copy()
                    factor_source = "Neut" # Shortened for sheet names
                    print(f"INFO: Using NEUTRALIZED factor '{factor_name}' for analysis.")
                elif not raw_factor_df.isna().all().all():
                    factor_to_analyze_df = raw_factor_df.copy()
                    factor_source = "Raw"
                    print(f"INFO: Using RAW factor '{factor_name}' for analysis.")
                else:
                    print(f"ERROR: No valid factor data (Raw or Neutralized) found for {factor_name}. Skipping analysis.")
                    continue

                # --- Prepare for Analysis ---
                analysis_performed = False
                # Factor display name for sheet naming - keep it concise
                factor_display_name = f"{factor_name[:15]}_{factor_source}" # Max 15 chars for factor part

                # Stack the chosen factor (Index=date, Columns=assets) -> Series (Index=(date, asset))
                factor_to_analyze_df.index.name = 'date'
                factor_to_analyze_df.columns.name = 'asset'
                factor_series = factor_to_analyze_df.stack(future_stack=True) # Use future_stack, dropna removed
                factor_series.index.names = ['date', 'asset']
                factor_series.rename('factor', inplace=True) # Ensure Series name is 'factor' for functions
                factor_series_clean = factor_series.dropna() # Drop NaNs *after* stacking

                # Align factor with forward returns (already stacked)
                aligned_data = pd.DataFrame()
                if factor_series_clean.empty:
                    print(f"ERROR: Factor series for {factor_display_name} is empty after dropna(). Skipping.")
                    continue
                else:
                    # Ensure forward returns are uniquely indexed if merging
                    fwd_returns_unique = forward_returns_df_stacked[~forward_returns_df_stacked.index.duplicated(keep='first')]
                    try:
                        # Merge the factor Series (as frame) with the forward returns DataFrame
                        aligned_data = pd.merge(factor_series_clean.to_frame(), fwd_returns_unique,
                                                left_index=True, right_index=True, how='inner')
                    except Exception as merge_err:
                        print(f"ERROR aligning data for {factor_display_name}: {merge_err}")
                        continue

                # Final check on aligned data
                # Drop rows where factor OR *any* of the analysis return periods are NaN
                aligned_data_clean = aligned_data.dropna(subset=['factor'] + analysis_periods_str, how='any')

                if aligned_data_clean.empty:
                    print(f"INFO: Skipping analysis for {factor_display_name} - no overlapping data.")
                    continue
                else:
                    print(f"Clean aligned data ready for {factor_display_name}. Shape: {aligned_data_clean.shape}")
                    # Identify return columns actually available after merge/dropna
                    available_ret_cols = [col for col in analysis_periods_str if col in aligned_data_clean.columns and not aligned_data_clean[col].isna().all()]
                    if not available_ret_cols:
                        print(f"ERROR: No valid forward returns columns remain for {factor_display_name}. Skipping.")
                        continue

                    # --- Initialize result containers ---
                    all_ic_summaries, all_daily_ics = [], {}
                    all_quantile_mean_rets, all_quantile_cum_rets = {}, {}
                    factor_daily_returns_dict, cumulative_factor_returns_dict = {}, {}
                    factor_analysis_summary = []

                    # --- Calculate IC Decay ---
                    ic_decay_results = pd.Series(dtype=float)
                    if fwd_returns_for_decay_dict:
                         # Pass the clean factor series (before alignment with specific returns)
                         ic_decay_results = calculate_ic_decay(factor_series_clean.copy(), fwd_returns_for_decay_dict, MAX_DECAY_LAG, method=ic_method)

                    # --- Calculate Quantile Turnover ---
                    all_quantile_turnover = pd.DataFrame()
                    print("\n--- Calculating Quantile Turnover ---")
                    # Pass factor from aligned data, only need 'factor' column
                    quantile_assignments = get_quantile_assignments(aligned_data_clean[['factor']].copy(), num_quantiles=num_quantiles)
                    if not quantile_assignments.empty and not quantile_assignments['quantile'].isna().all():
                        all_quantile_turnover = calculate_quantile_turnover(quantile_assignments, num_quantiles=num_quantiles)
                    else: print("WARN: Could not calculate turnover due to empty/NaN quantile assignments.")


                    # --- Loop through Analysis Periods ---
                    for ret_col in available_ret_cols:
                        print(f"\n===== Analyzing {factor_display_name} vs {ret_col} =====")
                        # Subset data needed for this specific return period
                        analysis_data_subset = aligned_data_clean[['factor', ret_col]].dropna()
                        if analysis_data_subset.empty:
                            print(f"INFO: No valid data for {factor_display_name} vs {ret_col} after dropna.")
                            continue

                        # --- Run Analyses ---
                        try: # Add try-except around individual analyses
                            mean_ret_q, cum_ret_q = quantile_analysis(analysis_data_subset.copy(), factor_display_name, num_quantiles=num_quantiles, ret_col=ret_col)
                            if mean_ret_q is not None: all_quantile_mean_rets[ret_col] = mean_ret_q
                            if cum_ret_q is not None: all_quantile_cum_rets[ret_col] = cum_ret_q

                            ic_summary, daily_ic = calculate_ic(analysis_data_subset.copy(), factor_display_name, ret_col=ret_col, method=ic_method)
                            if ic_summary is not None: all_ic_summaries.append(ic_summary)
                            if daily_ic is not None: all_daily_ics[ret_col] = daily_ic

                            factor_daily_ret, factor_cum_ret, _, factor_ann_stats = calculate_factor_returns(
                                analysis_data_subset.copy(), factor_display_name, ret_col=ret_col
                            )
                            if factor_daily_ret is not None: factor_daily_returns_dict[ret_col] = factor_daily_ret
                            if factor_cum_ret is not None: cumulative_factor_returns_dict[ret_col] = factor_cum_ret
                            if factor_ann_stats is not None: factor_analysis_summary.append(factor_ann_stats)

                            analysis_performed = True # Mark that at least one analysis ran
                        except Exception as e_analyze_period:
                             print(f"ERROR during analysis of {factor_display_name} vs {ret_col}: {e_analyze_period}")
                             traceback.print_exc() # Print detailed error for this period

                print(f"Analysis calculations finished. ({(time.time() - analysis_timer_start):.2f}s)")

                # --- Save Results ---
                if analysis_performed:
                    print(f"\n--- Saving results for {factor_display_name} to Excel ---")
                    save_timer_start = time.time()
                    # --- Save each result type using the helper defined outside the loop ---
                    if all_ic_summaries: sheets_saved_this_factor += save_to_excel_combined(pd.concat(all_ic_summaries, axis=1), 'IC_Sum', factor_display_name, writer)
                    if all_daily_ics: sheets_saved_this_factor += save_to_excel_combined(pd.concat(all_daily_ics, axis=1), 'IC_Daily', factor_display_name, writer)
                    if 'ic_decay_results' in locals() and not ic_decay_results.empty: sheets_saved_this_factor += save_to_excel_combined(ic_decay_results.to_frame(), 'IC_Decay', factor_display_name, writer)
                    if all_quantile_mean_rets: sheets_saved_this_factor += save_to_excel_combined(pd.concat(all_quantile_mean_rets, axis=1, join='outer').rename_axis('Quantile'), 'Q_MeanRet', factor_display_name, writer)
                    if all_quantile_cum_rets:
                        all_dfs_cum = []
                        for ret_p, cum_df in all_quantile_cum_rets.items():
                            if cum_df is not None and not cum_df.empty:
                                cum_df.columns.name = 'Quantile'; cum_df.columns = pd.MultiIndex.from_product([[ret_p], cum_df.columns], names=['Return_Period', 'Quantile'])
                                all_dfs_cum.append(cum_df)
                        sheets_saved_this_factor += save_to_excel_combined(pd.concat(all_dfs_cum, axis=1, join='outer') if all_dfs_cum else pd.DataFrame(), 'Q_CumRet', factor_display_name, writer)
                    if 'all_quantile_turnover' in locals() and not all_quantile_turnover.empty: sheets_saved_this_factor += save_to_excel_combined(all_quantile_turnover, 'Q_Turnover', factor_display_name, writer)
                    if factor_analysis_summary: sheets_saved_this_factor += save_to_excel_combined(pd.concat(factor_analysis_summary, axis=1).rename_axis('Metric'), 'Fctr_Stats', factor_display_name, writer)
                    if factor_daily_returns_dict: sheets_saved_this_factor += save_to_excel_combined(pd.concat(factor_daily_returns_dict, axis=1), 'Fctr_Ret', factor_display_name, writer)
                    if cumulative_factor_returns_dict: sheets_saved_this_factor += save_to_excel_combined(pd.concat(cumulative_factor_returns_dict, axis=1), 'Fctr_CumRet', factor_display_name, writer)

                    if sheets_saved_this_factor > 0:
                        print(f"--- Results for {factor_display_name} saved ({sheets_saved_this_factor} sheets). ({(time.time() - save_timer_start):.2f}s)---")
                        overall_success = True # Mark that at least one sheet was saved overall
                    else:
                        print(f"--- No data frames were valid for saving for {factor_display_name}. ---")

                else:
                    print(f"\n--- No analysis performed for factor '{factor_display_name}'. No results saved. ---")

                print(f"--- Factor {factor_name} processing time: {(time.time() - factor_timer_start):.2f}s ---")
            # --- End of loop through factors ---

            if overall_success:
                 print(f"\nAll factors processed. Finalizing Excel file: {combined_output_filename}")
            else:
                 print(f"\nWARNING: All factors processed, but no analysis results were generated or saved.")

            # ExcelWriter context manager handles saving on exit IF overall_success is True implicitly

    # --- End of Excel Writer context ---
    except ImportError:
        print("\nERROR: Could not prepare Excel file. `openpyxl` library not found.")
        print("Please install it: pip install openpyxl")
    except Exception as e_main_loop:
        print(f"\nERROR occurred during factor analysis loop or Excel writing: {e_main_loop}")
        traceback.print_exc()

# --- End of Script ---
print("\n=============================================")
print("=== Combined Factor Analysis Script Finished ===")
print("=============================================")

INFO: Start date localized to UTC: 2020-03-24 00:00:00+00:00
INFO: End date localized to UTC: 2025-03-24 00:00:00+00:00

--- Attempting to define universe based on index: ^HSI ---
Attempting to get constituents for ^HSI on 2020-03-24.
INFO: Using asset universe (Count: 83): ['0001.HK', '0002.HK', '0003.HK', '0005.HK', '0006.HK', '0011.HK', '0012.HK', '0016.HK', '0027.HK', '0066.HK']...
Using pandas_market_calendars for HK business days. Full fetch range index length: 1319
Target Analysis Date Range: 2020-03-24 00:00:00+00:00 to 2025-03-24 00:00:00+00:00 (1231 analysis days)

--- Downloading Price and Total Volume Data ---
Fetching data from 2019-12-16 to 2025-04-29 for 83 assets + benchmark ^HSI...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  83 of 83 completed


Asset price/volume data processed. Shape: (1319, 83)
Benchmark data processed. Length: 1319

--- Fetching Industry Classification Data ---


Fetching Industries: 100%|██████████| 83/83 [01:18<00:00,  1.05it/s]


Created Static Industry Dummies shape: (83, 50)

--- Loading/Defining Pre-calculated Factors ---
Attempting to load factors from: processed_alpha_data_values_1.csv
Successfully loaded factors from file. Initial shape: (102007, 15)
INFO: Loaded data appears to be in Format 2 (Stacked). Unstacking...
Successfully unstacked factors to Format 1 (Wide).
INFO: Localizing factor index timezone...
INFO: Converting factor index timezone...
Reindexing loaded factors to match analysis dates (1231) and assets (83)...
Final precalculated factors DataFrame ready. Shape: (1231, 1245)
Available factors: ['ATR_14d', 'Daily_High_Low_Range', 'EMA_20d', 'MA_Crossover_10_50', 'Mean_Reversion_20d', 'Moving_Average_Reversion', 'Normalized_BBW_20d_2std', 'Price_Momentum_10d', 'ROC_50d', 'RSI_14d', 'SMA_20d', 'Stochastic_Oscillator_14d', 'Trading_Volume', 'VROC_10d', 'Volume_Momentum_50d']

--- Calculating Style Factors (Beta, Size Proxy, Liquidity Proxy, Residual Volatility) ---
Calculating style factors usin

...


--- Calculating Quantile Turnover ---

===== Analyzing Trading_Volume_Raw vs 1D_fwd_ret =====

===== Analyzing Trading_Volume_Raw vs 3D_fwd_ret =====

===== Analyzing Trading_Volume_Raw vs 5D_fwd_ret =====
Analysis calculations finished. (15.70s)

--- Saving results for Trading_Volume_Raw to Excel ---
--- Results for Trading_Volume_Raw saved (9 sheets). (0.24s)---
--- Factor Trading_Volume processing time: 27.68s ---


==================== Processing Factor: VROC_10d ====================

--- Performing Factor Neutralization ---
Running neutralization regression day by day...


Neutralization completed for VROC_10d. (12.23s)

--- Starting Factor Analysis ---
INFO: Using RAW factor 'VROC_10d' for analysis.
Clean aligned data ready for VROC_10d_Raw. Shape: (98058, 4)
Calculating IC Decay...



--- Calculating Quantile Turnover ---

===== Analyzing VROC_10d_Raw vs 1D_fwd_ret =====

===== Analyzing VROC_10d_Raw vs 3D_fwd_ret =====

===== Analyzing VROC_10d_Raw vs 5D_fwd_ret =====
Analysis calculations finished. (15.48s)

--- Saving results for VROC_10d_Raw to Excel ---
--- Results for VROC_10d_Raw saved (9 sheets). (0.27s)---
--- Factor VROC_10d processing time: 27.98s ---


==================== Processing Factor: Volume_Momentum_50d ====================

--- Performing Factor Neutralization ---
Running neutralization regression day by day...


Neutralization completed for Volume_Momentum_50d. (12.04s)

--- Starting Factor Analysis ---
INFO: Using RAW factor 'Volume_Momentum_50d' for analysis.
Clean aligned data ready for Volume_Momentum_Raw. Shape: (95460, 4)
Calculating IC Decay...



--- Calculating Quantile Turnover ---

===== Analyzing Volume_Momentum_Raw vs 1D_fwd_ret =====

===== Analyzing Volume_Momentum_Raw vs 3D_fwd_ret =====

===== Analyzing Volume_Momentum_Raw vs 5D_fwd_ret =====
Analysis calculations finished. (15.14s)

--- Saving results for Volume_Momentum_Raw to Excel ---
--- Results for Volume_Momentum_Raw saved (9 sheets). (0.24s)---
--- Factor Volume_Momentum_50d processing time: 27.43s ---

All factors processed. Finalizing Excel file: factor_analysis_output_combined/combined_factor_analysis_results_1.xlsx

=== Combined Factor Analysis Script Finished ===


# Final version of LLM agent to decide which set of factor will be used {'HK_final_comprehensive_result_1.csv':'combined_factor_analysis_results.xlsx','HK_final_comprehensive_result_3.csv':'combined_factor_analysis_results_1.xlsx'}

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
OLLAMA_OPTIONS = {'temperature': 0.0, "seed": 42}

In [17]:
import re
import json
import time
import os
from ollama import Client
from tqdm import tqdm
import pandas as pd # Use pd alias consistently

# --- Configuration ---
OLLAMA_HOST = 'http://localhost:11434' # Adjust host if needed
MODEL_NAME = "llama3.2" # Define model name once
# --- LLM Generation Options for Determinism ---
OLLAMA_OPTIONS = {
    "temperature": 0.0,
    "seed": 42,  # Use a fixed seed for reproducibility
    # Add other options if needed/supported, e.g., top_p: 0.1
}

BASE_LOG_PATH = "./log/"
PROMPT_DIR = "./prompt_new_made/" # Corrected inconsistent hyphen/underscore
DATA_DIR = "./data/"
FACTOR_RESULTS_DIR = "./factor_analysis_output_combined/" # Assuming relative to script location
HK_STOCK_FILE = os.path.join(DATA_DIR, "HK_stock_earning_reports.xlsx") # Use os.path.join

# Initialize the Ollama client
try:
    client = Client(host=OLLAMA_HOST)
    # Optional: Check connection or model availability if needed
    # client.list()
except Exception as e:
    print(f"Error initializing Ollama client at {OLLAMA_HOST}: {e}")
    exit(1)


# Global variable for the current log file name (set within the loop)
log_file = "" # Initialized globally


# --- Remove Unused Functions ---
# def retry_until_expected(run, thread_id, expect): ...
# def get_last_text_message(thread_id): ...


# --- Helper Functions --- (Keep your improved versions from previous debug)

def log_to_file(type, message):
    """Logs input/output messages to the current log file."""
    global log_file
    if not log_file:
        print("Warning: log_file name not set before logging.")
        # Optionally set a default log file name here?
        # log_file = "default_comparison.log"
        return

    header = ""
    if type == "input":
        header = ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n"
    elif type == "output":
        header = "<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<\n"
    elif type == "info" or type == "error" or type == "warning":
         header = f"--- {type.upper()} ---\n"
    # else: header remains "" for plain messages

    full_log_path = os.path.join(BASE_LOG_PATH, log_file)
    os.makedirs(BASE_LOG_PATH, exist_ok=True)
    try:
        with open(full_log_path, "a", encoding="utf-8") as file:
            file.write(header + str(message) + "\n")
    except Exception as e:
        print(f"Error writing to log file {full_log_path}: {e}")


def read_file_content(file_path):
    """Safely reads content from a file."""
    try:
        # Detect encoding, common ones first
        encodings_to_try = ['utf-8', 'gbk', 'latin-1']
        content = None
        for enc in encodings_to_try:
            try:
                with open(file_path, "r", encoding=enc) as f:
                    content = f.read()
                break
            except UnicodeDecodeError:
                continue
            except Exception as e:
                return f"Error reading file {file_path}: {e}"

        if content is None:
             return f"Could not decode file {file_path} with tried encodings."
        return content

    except FileNotFoundError:
        return f"Error: File not found at {file_path}"
    except Exception as e:
        return f"Error reading file {file_path}: {e}"


def get_excel_data_as_string(file_path, max_rows=20):
    """Reads an Excel file and returns its content as a string."""
    try:
        # Use the imported pandas as pd
        df = pd.read_excel(file_path) # Removed hardcoded sheet_name unless necessary
        return f"Data from {os.path.basename(file_path)} (Headers and first {max_rows} rows):\n{df.head(max_rows).to_string()}"
    except FileNotFoundError:
        return f"Error: Excel file not found at {file_path}"
    except Exception as e:
        return f"Error reading Excel file {file_path}: {e}"

# --- Core Comparison Logic ---

def compare(index1_path, index2_path):
    """
    Compares two index files using Ollama with deterministic settings.
    Returns "1" or "2".
    """
    print(f"\n--- Comparing ---")
    print(f"Index 1: {os.path.basename(index1_path)}")
    print(f"Index 2: {os.path.basename(index2_path)}")
    log_to_file("info", f"Comparing {os.path.basename(index1_path)} vs {os.path.basename(index2_path)}")

    # --- Load Prompts ---
    # Use os.path.join for robustness
    instruction = read_file_content(os.path.join(PROMPT_DIR, "0-instruction.md"))
    preamble = read_file_content(os.path.join(PROMPT_DIR, "1-preamble.md"))
    file_structure_prompt = read_file_content(os.path.join(PROMPT_DIR, "2-file_structure.md"))
    index1_prompt = read_file_content(os.path.join(PROMPT_DIR, "3-index1.md"))
    index2_comparison_prompt_template = read_file_content(os.path.join(PROMPT_DIR, "4-index2.md"))

    # Basic check if prompts loaded correctly (can be more robust)
    if any(p.startswith("Error:") for p in [instruction, preamble, file_structure_prompt, index1_prompt, index2_comparison_prompt_template]):
         print("Error loading one or more prompt files. Check paths and file existence.")
         log_to_file("error", "Failed to load one or more prompt files.")
         # Decide how to handle this - maybe return a default or raise exception
         return "1" # Or None, or raise error

    # --- Build Conversation History ---
    thread_messages = []
    # Optional: Add instruction as system message if appropriate
    # if instruction and not instruction.startswith("Error:"):
    #    thread_messages.append({"role": "system", "content": instruction})

    # Function to handle chat calls and logging consistently
    def run_chat_step(user_content, log_input_summary=None):
        nonlocal thread_messages # Allow modifying the outer scope variable
        if log_input_summary:
            log_to_file("input", log_input_summary)
        else:
             log_to_file("input", user_content[:500] + "...") # Log beginning of long content

        thread_messages.append({"role": "user", "content": user_content})
        try:
            # *** Apply deterministic options here ***
            response = client.chat(
                model=MODEL_NAME,
                messages=thread_messages,
                options=OLLAMA_OPTIONS # Pass the options dictionary
            )
            response_content = response['message']['content'].strip()
            log_to_file("output", response_content)
            thread_messages.append(response['message']) # Add assistant response to history
            return response_content
        except Exception as e:
            print(f"Error during Ollama chat step: {e}")
            log_to_file("error", f"Ollama chat error: {e}")
            return None # Indicate error

    # 1. Preamble
    if run_chat_step(preamble) is None: return "1" # Handle error, return default

    # 2. File Structure & HK Data
    hk_data_str = get_excel_data_as_string(HK_STOCK_FILE)
    content_with_hk_data = f"{file_structure_prompt}\n\n{hk_data_str}"
    if run_chat_step(content_with_hk_data, log_input_summary=f"{file_structure_prompt[:100]}...\n+ HK Data from {os.path.basename(HK_STOCK_FILE)}") is None: return "1"

    # 3. Index 1 Info (Prompt only)
    prompt_for_index1 = f"{index1_prompt}\n\nFile Reference: {os.path.basename(index1_path)}"
    if run_chat_step(prompt_for_index1, log_input_summary=f"{index1_prompt[:100]}...\n+ Ref: {os.path.basename(index1_path)}") is None: return "1"

    # 4. Index 2 & Comparison Request
    index1_content = read_file_content(index1_path)
    index2_content = read_file_content(index2_path)

    if index1_content.startswith("Error:") or index2_content.startswith("Error:"):
        print(f"Error reading index files: {index1_content} / {index2_content}")
        log_to_file("error", f"Error reading index files:\n1: {index1_content}\n2: {index2_content}")
        return "1" # Default on file read error

    # Construct the final comparison prompt
    comparison_prompt = f"""{index2_comparison_prompt_template}

    --- Data for Index 1 ({os.path.basename(index1_path)}) ---
    {index1_content}
    --- End of Data for Index 1 ---

    --- Data for Index 2 ({os.path.basename(index2_path)}) ---
    {index2_content}
    --- End of Data for Index 2 ---

    YOU MUST NOT GENERATE ANY PYTHON CODES.

    Based on all the information provided (including previous context about file structures, HK data, and the data for the two indices above), which index represents the better alpha strategy?

    Please respond with only the single digit "1" if {os.path.basename(index1_path)} is better, or the single digit "2" if {os.path.basename(index2_path)} is better. Your response must contain *only* the number and nothing else.
    """

    # Run final comparison chat
    final_response_content = run_chat_step(comparison_prompt, log_input_summary=f"{index2_comparison_prompt_template[:100]}...\n+ Data for {os.path.basename(index1_path)} & {os.path.basename(index2_path)}\n+ Instruction: Respond '1' or '2'")
    if final_response_content is None: return "1" # Handle error

    # --- Extract Decision (Stricter check first) ---
    if final_response_content == "1":
        index = "1"
    elif final_response_content == "2":
        index = "2"
    else:
        # Fallback to regex if the strict check fails
        index_match = re.search(r"\b(1|2)\b", final_response_content)
        if index_match:
            index = index_match.group(1)
            warning_msg = f"LLM response ('{final_response_content}') was not strictly '1' or '2'. Extracted '{index}' using regex."
            print(f"Warning: {warning_msg}")
            log_to_file("warning", warning_msg)
        else:
            error_msg = f"Could not determine the better index from the response: '{final_response_content}'. Defaulting to 1."
            print(f"Error: {error_msg}")
            log_to_file("error", error_msg)
            index = "1" # Default to 1 if unsure

    log_to_file("info", f"Selected better alpha index: {index} ({os.path.basename(index1_path if index == '1' else index2_path)})")
    print(f"LLM chose index: {index}")
    return index

# --- Main Execution ---

if __name__ == "__main__":
    if not os.path.isdir(FACTOR_RESULTS_DIR):
        print(f"Error: Factor results directory not found: {FACTOR_RESULTS_DIR}")
        exit(1)

    files = os.listdir(FACTOR_RESULTS_DIR)
    files = [f for f in files if f.lower().endswith((".xlsx", ".csv", ".txt")) and not f.startswith('~$')]

    if not files:
        print(f"Error: No suitable files found in {FACTOR_RESULTS_DIR}")
        exit(1)

    # *** Sort the files for consistent comparison order ***
    files.sort()
    print(f"Found {len(files)} files to compare in {FACTOR_RESULTS_DIR}. Order: {files}") # Show sorted order

    best_file_name = files[0]
    best_file_original_index = 0 # Track original index in the *sorted* list

    # Use enumerate starting from 1 for the comparison loop
    for i, next_file_name in enumerate(tqdm(files[1:], desc="Comparing Files")):
        round_num = i + 1
        # Find original index of the 'next' file IN THE SORTED LIST
        next_file_original_index = i + 1 # Since files[0] is index 0, files[1] is index 1 etc.

        # Update global log file name for this comparison
        log_file = f"round-{round_num}-{best_file_original_index}_vs_{next_file_original_index}.log"
        print(f"\nRound {round_num}: Comparing '{best_file_name}' (Sorted Index {best_file_original_index}) vs '{next_file_name}' (Sorted Index {next_file_original_index})")
        print(f"Logging to: {os.path.join(BASE_LOG_PATH, log_file)}")

        current_best_path = os.path.join(FACTOR_RESULTS_DIR, best_file_name)
        next_file_path = os.path.join(FACTOR_RESULTS_DIR, next_file_name)

        # Basic check if files exist before comparing (robustness)
        if not os.path.exists(current_best_path) or not os.path.exists(next_file_path):
             print(f"Error: One or both files not found for comparison: {current_best_path}, {next_file_path}")
             log_to_file("error", f"Comparison skipped: File(s) not found - Best: {current_best_path}, Next: {next_file_path}")
             continue # Skip this comparison round

        selected_index = compare(current_best_path, next_file_path)

        if selected_index == "2":
            print(f"Result: '{next_file_name}' selected as new best.")
            log_to_file("info", f"'{next_file_name}' selected as new best over '{best_file_name}'.")
            best_file_name = next_file_name
            # Update the index of the best file to the index of the winner (next file)
            best_file_original_index = next_file_original_index
        elif selected_index == "1":
             print(f"Result: '{best_file_name}' remains the best.")
             log_to_file("info", f"'{best_file_name}' remains best over '{next_file_name}'.")
             # best_file_original_index remains unchanged
        else:
            # Handle potential error case from compare function if it returned None or similar
            print(f"Warning: Comparison between {best_file_name} and {next_file_name} resulted in unexpected value '{selected_index}'. Keeping current best.")
            log_to_file("warning", f"Comparison between {best_file_name} and {next_file_name} resulted in unexpected value '{selected_index}'. Keeping current best.")
            # Keep the current best_file, best_file_original_index remains unchanged


    print("\n" + "="*30)
    print(f"Comparison finished.")
    print(f"The best alpha file identified (based on sorted order comparison) is: {best_file_name}")
    # Optional: map back to original unsorted index if needed, but sorted index is more meaningful for the run
    print(f"This file had index {best_file_original_index} in the alphabetically sorted list.")
    print("="*30)

Found 2 files to compare in ./factor_analysis_output_combined/. Order: ['combined_factor_analysis_results.xlsx', 'combined_factor_analysis_results_1.xlsx']


Comparing Files:   0%|          | 0/1 [00:00<?, ?it/s]


Round 1: Comparing 'combined_factor_analysis_results.xlsx' (Sorted Index 0) vs 'combined_factor_analysis_results_1.xlsx' (Sorted Index 1)
Logging to: ./log/round-1-0_vs_1.log

--- Comparing ---
Index 1: combined_factor_analysis_results.xlsx
Index 2: combined_factor_analysis_results_1.xlsx


Comparing Files: 100%|██████████| 1/1 [00:45<00:00, 45.43s/it]

LLM chose index: 2
Result: 'combined_factor_analysis_results_1.xlsx' selected as new best.

Comparison finished.
The best alpha file identified (based on sorted order comparison) is: combined_factor_analysis_results_1.xlsx
This file had index 1 in the alphabetically sorted list.
